# <center>GMD_Manuscript_Figures</center>
This jupyter-notebook is used to make the GMD methods paper figures






# F01: Reconstructed Bathymetry plots and Basin Boundaries Used in Bogumil et al. (2024) (See FinalizedFigures folder for Composite Figure)

In [ ]:
#################################################
#################### Imports ####################
#################################################

# For reading matlab files that store plate boundaries.
import scipy.io
import os

# For plotting
from netCDF4 import Dataset
import copy as cp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from shapely.geometry import Polygon
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature

###################################
## Define Functions for Figure  ###
###################################
def load_netcdf_data(age):
    # Read bathymetry file.
    ds = Dataset(os.getcwd()+'/PNAS_Bogumil_Results/bathymetryNCFiles/Bathymetry_{}Ma.nc'.format(age))

    # Assign data to variables
    lon, lat = np.meshgrid(ds['lon'][:], ds['lat'][:]);
    z = ds['z'][:].data;
    continents = ds['z'][:].mask;
    continentsnew = cp.deepcopy(continents).astype(float)
    continentsnew[continentsnew==0] = np.nan

    
    # Close dataset
    ds.close();
    
    # Return variables
    return lon, lat, z, continentsnew

def remove_non_unique_rows(arr):
    # Count occurrences of each row
    unique_rows, counts = np.unique(arr, axis=0, return_counts=True)
    # Only keep rows that appear once
    unique_only = unique_rows[counts == 1]
    return unique_only

def readBoundaries(age, pacificValues=[1,0]):
    # Load the .mat file
    mat_data = scipy.io.loadmat(os.getcwd()+'/PNAS_Bogumil_Results/basinPolygons/Earth{}Ma_basin_poly.mat'.format(age))

    # Print variable names in the file (ignoring MATLAB metadata entries)
    variable_names = [key for key in mat_data.keys() if not key.startswith('__')]
    #print("Variables in .mat file:", variable_names);

    # Access a specific variable (replace 'your_variable_name' as needed)
    boundaries = mat_data['Earth{}Ma'.format(age)][0][0];
    
    # Merge two Pacific boundaries
    if pacificValues is not None:
        x = np.vstack( (boundaries[pacificValues[0]], boundaries[pacificValues[1]]) )
        x = remove_non_unique_rows(x)
        boundariesOut = np.array(np.arange(len(boundaries)-1), dtype=object)
        boundariesOut[0] = x

        cnt = 0;
        for i in range(len(boundaries)):
            #print(i, (pacificValues[0]==i), (pacificValues[1]==i), ((pacificValues[0]==i) | (pacificValues[1]==i)))
            if not ((pacificValues[0]==i) | (pacificValues[1]==i)):
                #print(i)
                boundariesOut[cnt+1] = boundaries[i]
                boundariesOut[cnt+1] = boundaries[i]
                cnt+=1;
    else:
        boundariesOut = np.array(np.arange(len(boundaries)), dtype=object)
        for i in range(len(boundaries)):
            boundariesOut[i] = boundaries[i]
        

    return boundariesOut



############################################################################
# Read and plot bathymetry and basin boundaries from Bogumil et al. (2024) #
############################################################################

# Load NetCDF file
age1 = 0; age2 = 60;
lon, lat, z1, continents1 = load_netcdf_data(age1)
lon, lat, z2, continents2 = load_netcdf_data(age2)

# Load basin boundaries
boundaries1 = readBoundaries(age1, None)
boundaries2 = readBoundaries(age2, None)

# Reduce resolution
factor = 1
lon = lon[::factor].T[::factor].T
lat = lat[::factor].T[::factor].T
z1  = z1[::factor].T[::factor].T
z2  = z2[::factor].T[::factor].T
continents1 = continents1[::factor].T[::factor].T
continents2 = continents2[::factor].T[::factor].T


# Create the figure
## Set up projection and figure
proj = ccrs.Robinson()
fig, axs = plt.subplots(
    2, 1,
    figsize=(12, 10),
    subplot_kw={'projection': proj},
    constrained_layout=False
)

# Define common color scale
vmin = np.nanmin([np.nanmin(z1), np.nanmin(z2)])
vmax = np.nanmax([np.nanmax(z1), np.nanmax(z2)])
vmin = 0
vmax = 6000


# Example polygon (replace with your actual lat/lon)
poly_lats = [-10, -10, 0, 0, -10]
poly_lons = [100, 110, 110, 100, 100]
polygon = Polygon(zip(poly_lons, poly_lats))

# Sample reversed blue cmap for bathymetry
N = 35
N = 30
blues_cm = mpl.colormaps['Blues'].resampled(N)

# Plot first dataset
## Plot bathymetry
mesh11 = axs[0].pcolormesh(lon, lat, z1, transform=ccrs.PlateCarree(), cmap=blues_cm, shading='auto', vmin=vmin, vmax=vmax)
## Plot Land
mesh12 = axs[0].pcolormesh(lon, lat, continents1, transform=ccrs.PlateCarree(), cmap='YlOrBr', shading='auto', vmin=0, vmax=4)
## Plot basin boundaries
colors = ['black', 'black', 'black', 'black', 'black']

#colors = ['black', 'black', 'red', 'green']
cnt=0
for boundary in boundaries1:
    lonB = boundary[:,0]
    latB = -1*boundary[:,1]
    latB[latB > 90] = 90;
    latB[latB < -90] = -90;
    lonB[lonB > 180] = 180;
    lonB[lonB < -180] = -180;

    polygon1 = Polygon(zip(lonB, latB))
    colori = colors[cnt]
    axs[0].plot(lonB, latB, linestyle='solid', color=colori, transform=ccrs.PlateCarree())
    cnt+=1;


# Plot second dataset
## Plot bathymetry
mesh21 = axs[1].pcolormesh(lon, lat, z2, transform=ccrs.PlateCarree(), cmap=blues_cm, shading='auto', vmin=vmin, vmax=vmax)
## Plot Land
mesh22 = axs[1].pcolormesh(lon, lat, continents2, transform=ccrs.PlateCarree(), cmap='YlOrBr', shading='auto', vmin=0, vmax=4)
## Plot basin boundaries
cnt=0
for boundary in boundaries2:
    lonB = boundary[:,0]
    latB = -1*boundary[:,1]
    
    latB[latB > 90] = 90;
    latB[latB < -90] = -90;
    lonB[lonB > 180] = 180;
    lonB[lonB < -180] = -180;

    polygon1 = Polygon(zip(lonB, latB))
    colori = colors[cnt]
    axs[1].plot(lonB, latB, linestyle='solid', color=colori, transform=ccrs.PlateCarree())
    #axs[1].add_geometries([polygon1], crs=ccrs.PlateCarree(), facecolor='none', edgecolor=colori, linewidth=2)
    cnt+=1;


# Add shared colorbar
cbar_ax = fig.add_axes([0.25, 0.05, 0.5, 0.02])  # [left, bottom, width, height]
cbar = fig.colorbar(mesh21, cax=cbar_ax, orientation='horizontal')
cbar.set_label("Bathymetry")

# Save figure
directory_name = os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/Bogumil_etal_2024_Bathymetry_and_BasinBoundaries"
os.makedirs(directory_name, exist_ok=True)

plt.savefig(directory_name+"/0_60Ma_Bathymetry_and_BasinBoundaries.png", dpi=600, transparent=True)


# F04: Diagrams used to construct Girvan-Newman and Louvain Algorithm Diagram Figure (See FinalizedFigures folder for Composite Figure)

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx

# Create the directory if it doesn't exist
directory_name = os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/GN_L_Algorithm";
os.makedirs(directory_name, exist_ok=True)


###################################
## Define Functions for Figure 2 ##
###################################
class NeighborNetwork():
    """
    createNeighborNetwork is a function designed
    to create an networkx spatial graph with neighbor
    node edges. Node edges are given a value of 1,
    but can be modified using a method.
    
    """
    
    def __init__(self, size=(2,8)):
        """
        Initializes NeighborNetwork.
        
        Parameters
        -----------
        size : TUPLE
            A tuple describing the size of the output
            graph. The graph will have size[0]*size[1]
            nodes.
        """
        # Construct new graph
        self.G = nx.Graph()
        
        # Add nodes
        ## Create numpy array of input size
        rows = size[0]
        cols = size[1]
        self.XX, self.YY = np.meshgrid(np.arange(cols), np.arange(rows))
        
        # Create nodes
        self.pos = [];
        for x, y in zip(self.XX.flatten(), self.YY.flatten()):
            currentNodeid = x + y*cols;
            self.G.add_node(currentNodeid, pos=(x-(cols-1)/2,
                                                (y-(rows-1)/2)*np.abs(0.4*(x-(cols-1)/2))+(y-(rows-1)/2))
                           )
            self.pos.append(currentNodeid)
        self.pos=np.array(self.pos);
        
        # Iterate over each nodes' xy position adding edges
        # adding each edge.
        for x, y in zip(self.XX.flatten(), self.YY.flatten()):
            # Positions
            #               X + Y*(# of columns) 
            currentNodeid = x + y*cols;
            leftid        = x + y*cols - 1;
            aboveid       = x + (y-1)*cols
            
            # Add edges to other nodes            
            if (x > 0):
                # Make connection left
                self.G.add_edge(currentNodeid,leftid, weight=1)            
            if (y > 0):
                # Make connection below
                self.G.add_edge(currentNodeid,aboveid, weight=1)
                
            # Add edges to other nodes            
            if (x == 0):
                # Make connection left
                self.G.add_edge(currentNodeid,leftid+cols, weight=1)            
                
                
    def plot(self, title="Initial network", saveFig=True):
        elarge = [(u, v) for (u, v, d) in self.G.edges(data=True) if d["weight"] > .5]
        esmall = [(u, v) for (u, v, d) in self.G.edges(data=True) if d["weight"] <= .5]
        
        pos = nx.get_node_attributes(myNeighborNetwork.G, "pos")
        
        # Create new figure
        fig = plt.figure()

        # nodes
        nx.draw_networkx_nodes(self.G, pos, node_size=700)

        # edges
        nx.draw_networkx_edges(self.G, pos, edgelist=elarge, width=6)
        nx.draw_networkx_edges(
            self.G, pos, edgelist=esmall, width=6, alpha=0.5, edge_color="r", style="dashed"
        )

        # node labels
        nx.draw_networkx_labels(self.G, pos, font_size=20, font_family="sans-serif")
        # edge weight labels
        edge_labels = nx.get_edge_attributes(self.G, "weight")
        nx.draw_networkx_edge_labels(self.G, pos, edge_labels)

        ax = plt.gca()
        plt.title(title)
        ax.margins(.1)
        plt.axis("off")
        plt.tight_layout()
        if saveFig:
            plt.savefig(directory_name+"/"+title+".svg", dpi=600)

# Girvan-newman algorithm 
def most_central_edge(G):
    centrality = nx.edge_betweenness_centrality(G, weight="weight")
    return max(centrality, key=centrality.get)

# Function that removes highest between edges removeNEdges times.
def girvanNewmanBetweennessDiagram(removeNEdges=1):
    # Make graph ()
    myNeighborNetwork = NeighborNetwork(size=(2,4))
    ## Change edge attributes
    myNeighborNetwork.G.edges[5,6]['weight'] = .5
    myNeighborNetwork.G.edges[1,2]['weight'] = .7

    # Plot again (removed most central edge)
    for i in range(removeNEdges):
        u,v = most_central_edge(myNeighborNetwork.G)
        centrality = nx.edge_betweenness_centrality(myNeighborNetwork.G, weight="weight")
        print("Removing edge from node pair {}-{}".format(u,v))
        myNeighborNetwork.G.remove_edge(u,v)
        
        
    myNeighborNetwork.plot(title="3-{} Removed highest edge betweenness centrality - Girvan-Newman".format(i))


    # Plot again (change edge values to centrality values)

    ## Set node edges to have betweenness centrality value
    list(centrality.values())/ np.min(list(centrality.values()))
    for nodes in centrality:
        myNeighborNetwork.G.add_edge(nodes[0],
                                     nodes[1],
                                     weight=centrality[nodes]/np.min(list(centrality.values())))

    myNeighborNetwork.plot(title="2-{} Betweenness Centrality Values - Girvan-Newman".format(i))
        

###############################
### Girvan-newman algorithm ###
###############################
# Make graph ()
myNeighborNetwork = NeighborNetwork(size=(2,4))
## Change edge attributes
myNeighborNetwork.G.edges[5,6]['weight'] = .5
myNeighborNetwork.G.edges[1,2]['weight'] = .7

# Plot graph
myNeighborNetwork.plot(title='1 Initial network - Girvan-Newman')


#############################################
### Girvan-newman algorithm removed edges ###
#############################################
# Remove one edge (1 community)
girvanNewmanBetweennessDiagram(1)
# Remove two edge (1 community)
girvanNewmanBetweennessDiagram(2)
# Remove three edge (1 community)
girvanNewmanBetweennessDiagram(3)
# Remove four edge (Creates 2 communities)
girvanNewmanBetweennessDiagram(4)


###############################
###### Louvain algorithm ######
###############################
# Make graph ()
myNeighborNetwork = NeighborNetwork(size=(2,4))

commL = nx.community.louvain_communities(myNeighborNetwork.G, weight='weight', max_level=20)

## Change edge attributes
# myNeighborNetwork.G.remove_edge(4,5)
# myNeighborNetwork.G.remove_edge(5,6)
# myNeighborNetwork.G.remove_edge(6,7)
# myNeighborNetwork.G.remove_edge(7,4)

# myNeighborNetwork.G.remove_edge(0,1)
# myNeighborNetwork.G.remove_edge(1,2)
# myNeighborNetwork.G.remove_edge(2,3)
# myNeighborNetwork.G.remove_edge(3,0)

# myNeighborNetwork.G.remove_edge(4,0)
# myNeighborNetwork.G.remove_edge(5,1)
# myNeighborNetwork.G.remove_edge(6,2)
# myNeighborNetwork.G.remove_edge(7,3)

###################################
## Define Functions for Figure 2 ##
###################################
def remove_intercommunity_edges(G, communities):
    """
    Removes all edges between communities in the graph G.

    Parameters:
    - G: A NetworkX graph (can be directed or undirected)
    - communities: A list of sets, each set contains nodes in one community
                   (e.g., output of nx.community.louvain_communities)

    Returns:
    - G_sub: A copy of G with only intra-community edges retained
    """
    # Create a copy to avoid modifying the original graph
    G_sub = G.copy()
    
    # Build a node to community index map
    node_to_comm = {}
    for idx, comm in enumerate(communities):
        for node in comm:
            node_to_comm[node] = idx

    # Identify and remove inter-community edges
    edges_to_remove = []
    for u, v in G.edges():
        if node_to_comm.get(u) != node_to_comm.get(v):
            edges_to_remove.append((u, v))

    G_sub.remove_edges_from(edges_to_remove)
    return G_sub


###############################
###### Louvain algorithm ######
###############################
# Make graph ()
myNeighborNetwork1 = NeighborNetwork(size=(2,4))
myNeighborNetwork2 = NeighborNetwork(size=(2,4))

## Change edge attributes
myNeighborNetwork1.G.edges[5,6]['weight'] = .5
myNeighborNetwork1.G.edges[1,2]['weight'] = .7
myNeighborNetwork2.G.edges[5,6]['weight'] = .5
myNeighborNetwork2.G.edges[1,2]['weight'] = .7

## Find communities
seed = 3
commL1 = nx.community.louvain_communities(myNeighborNetwork1.G, weight='weight', threshold=.9, max_level=100, seed=seed)
commL2 = nx.community.louvain_communities(myNeighborNetwork2.G, weight='weight', threshold=.2, max_level=100, seed=seed)

## Find Louvain communities
modularity1 = nx.community.modularity(myNeighborNetwork1.G, commL1, weight='weight')
modularity2 = nx.community.modularity(myNeighborNetwork2.G, commL2, weight='weight')

# Plot graph - Initial graph
myNeighborNetwork1.plot(title='4 Initial network - Louvain', saveFig=True)

# Plot graph - Once reduced
## Removed all edge connections and add them where there are nodes forming communities
myNeighborNetwork1.G = remove_intercommunity_edges(myNeighborNetwork1.G, commL1)
## Plot
myNeighborNetwork1.plot(title='5 Once Reduced - Louvain - seed {}'.format(seed), saveFig=True)

# # Plot graph - Twice reduced
## Removed inter-community connections
myNeighborNetwork2.G = remove_intercommunity_edges(myNeighborNetwork2.G, commL2)
## Plot
myNeighborNetwork2.plot(title='6 Twice Reduced - Louvain - seed {}'.format(seed), saveFig=True)





# F07: Prepare For Analysis by Downloading Etopo

In [ ]:
# Import necessary libraries
import ExoCcycle as EC
import matplotlib as mpl
import numpy as np
import os

# Create bathymetry object with body=... attributes
# Try setting body='mars' | 'earth' | 'moon', 'Venus'
body="Earth"
planetBathy = EC.Bathymetry.BathyMeasured(body=body)

# Create the directory if it doesn't exist
directory_name = os.getcwd()+"/topographies/{}".format(planetBathy.model)
os.makedirs(directory_name, exist_ok=True)

# Download topography model (note that is must only be done once per body).
# NOTE: UNCOMMENT THIS SECTION TO DOWNLOAD TOPOGRAPHY MODEL
planetBathy.getTopo(os.getcwd(),
                  verbose=True);

# Read topography. This method will generate a topography model netCDF4 file with 
# new_resolution, in degrees. Note that the generated topography model will be 
# cell registered (All calculations from here on out are in cell registered to
# simplify codes and reduce data loss on conversions).
planetBathy.readTopo(os.getcwd(),
                   new_resolution=1,
                   verbose=False);

# Generate a bathymetry model base on a set of input methods and properties. The setSeaLelvel
# method has multiple ways to fill topography with oceans. The two currently implemented at
# the time of this JN creation are as follows:
#     1) basinVolume : An option to define bathymetry by flooding topography with
#                      basinVolume['uncompactedVol'] amount of ocean water, in m3.
#     2) OceanArea : Option to define bathymetry by flooding topography until
#                    oceanArea['area'], decimal percent, of global area is covered
#                    with oceans.
# 
# Here, I use the OceanArea constraint and block the basinVolume constraint with comments.
# 
planetBathy.setSeaLevel(basinVolume = {"on":False, 'uncompactedVol':None},
                      oceanArea = {"on":True, "area":0.7},
                      isostaticCompensation = {"on":False}, verbose=False)

# Note that in this basinVolume example the uncompactedVol was set to the oceanArea
# bathymetry model's VOC, where oceanArea = {"on":True, "area":0.7}. 
#
# planetBathy.setSeaLevel(basinVolume = {"on":True, 'uncompactedVol':3.299187952154623e+17},
#                       oceanArea = {"on":False, "area":0.7},
#                       isostaticCompensation = {"on":False}, verbose=False)


# Save bathymetry model as netCDF4. Note that models will be saved under the same root folder
# that was supplied to the readTopo(...) method.
# 
# only the bathymetry array is saved in this file. Other relevant values are represented
# as vectors (e.g., lat, lon, area-weights, global bathymetery distributions, etc) or
# attributes (e.g., VOC, AOC, high latitude cutoff, etc).
# 
planetBathy.saveBathymetry()

#
# To see a full set of stored netCDF4 values, the user can uncomment the code below
# 
# planetBathy.readBathymetry()
# print(planetBathy.bathync)
# print(planetBathy.bathync.variables)

# Plot bathymetry model
blues_cm = mpl.colormaps['Blues'].resampled(100)
EC.plotHelper.plotGlobal(planetBathy.lat, planetBathy.lon, planetBathy.bathymetry,
                    outputDir = planetBathy.data_dir+"/bathymetries/{}".format(planetBathy.model),
                    fidName = "{}-setSeaLevel_Area0p7.png".format(planetBathy.model),
                    cmapOpts={"cmap":blues_cm,
                              "cbar-title":"cbar-title",
                              "cbar-range":[np.nanmin(np.nanmin(planetBathy.bathymetry)),
                                            np.nanmean(planetBathy.bathymetry)+2*np.nanstd(planetBathy.bathymetry)]},
                    pltOpts={"valueType": "Bathymetry",
                             "valueUnits": "m",
                             "plotTitle":"{}".format(planetBathy.model),
                             "plotZeroContour":True},
                    savePNG = True)


# Plot bathymetry model w/ bathymetry histograms
EC.plotHelper.plotGlobalwHist(planetBathy.lat, planetBathy.lon, planetBathy.bathymetry,
                        planetBathy.binEdges, planetBathy.bathymetryAreaDist_wHighlat, planetBathy.bathymetryAreaDist, planetBathy.highlatlat,
                        outputDir = planetBathy.data_dir+"/bathymetries/{}".format(planetBathy.model),
                        fidName = "{}-setSeaLevel_Area0p7.png".format(planetBathy.model),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[np.nanmin(np.nanmin(planetBathy.bathymetry)),
                                                np.nanmean(planetBathy.bathymetry)+2*np.nanstd(planetBathy.bathymetry)]},
                        pltOpts={"valueType": "Bathymetry",
                                 "valueUnits": "m",
                                 "plotTitle":"{}".format(planetBathy.model),
                                 "plotZeroContour":True},
                        savePNG = True)



# F07: (Leiden Ensemble - Reduction On Etopo Bathymetry)

Makes two subplots:
- Etopo bathymetry map
- Leiden detected communities

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset


#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden"

# Define basin merging criteria
mergerPackageName = 'None'; # ['threshold'] = [0]

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    print(resolution, ensembleSize, minBasinCnt)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":False,
                       "mergerPackage":mergerPackage,
                       "njobs":4}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results    
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/CompositeMethod_Leiden')
    print("Storing images in {}".format(fldName))

    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));



    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_1.0deg.nc".format(body),
                             body=body);


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)
    
    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)



    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
        



    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)

    #########################
    ### Plot input fields ###
    #########################
    plotRange = [0, 6000];
    N = 1000
    blues_cm = mpl.colormaps['Blues'].resampled(N)
    #plotRange = [ave-1*std, ave+1*std];
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "boundaryColor":'k',
                                 "boundaryLinewidth":1.5,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    


# F07: (Leiden Ensemble - Reduction and Girvan-Newman On Etopo Bathymetry)

Makes two subplots:
- Etopo bathymetry map
- Leiden-Girvan-Newman detected communities

Note that mergeSmallBasins is set to false, such that we can see the effect of Girvan-Newman mergers.
- basins.detectionMethod["mergerPackage"]["mergeSmallBasins"]["on"]=False

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset


#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden-Girvan-Newman"

# Define basin merging criteria
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    print(resolution, ensembleSize, minBasinCnt)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}
    
    # Make folder to hold figure results
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/CompositeMethod_Leiden-GirvanNewman')
    print("Storing images in {}".format(fldName))

    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));


    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_1.0deg.nc".format(body),
                             body=body);


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    # Note that mergeSmallBasins is set to false, such that we can see
    # the effect of Girvan-Newman mergers.
    mergerPackage["mergeSmallBasins"]["on"]=False
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)
    
    
    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)


    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)



    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)

    #########################
    ### Plot input fields ###
    #########################
    plotRange = [0, 6000];
    N = 1000
    blues_cm = mpl.colormaps['Blues'].resampled(N)
    #plotRange = [ave-1*std, ave+1*std];
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "boundaryColor":'k',
                                 "boundaryLinewidth":1.5,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    


# F07: (Leiden Ensemble - Reduction and Post-Processing (small basin merger) On Etopo Bathymetry)

Makes two subplots:
- Etopo bathymetry map
- Leiden detected communities with merged small communities

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset

# Create nodeclustering object
from cdlib import evaluation
from cdlib import NodeClustering

#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden"

# Define basin merging criteria
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [0.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    print(resolution, ensembleSize, minBasinCnt)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage,
                       "njobs":4}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/CompositeMethod_Leiden-PP')
    print("Storing images in {}".format(fldName))
    
    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));


    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_1.0deg.nc".format(body),
                             body=body);


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)


    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)



    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)


    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)

    #########################
    ### Plot input fields ###
    #########################
    plotRange = [0, 6000];
    N = 1000
    blues_cm = mpl.colormaps['Blues'].resampled(N)
    #plotRange = [ave-1*std, ave+1*std];
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "boundaryColor":'k',
                                 "boundaryLinewidth":1.5,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    
    # Create node cluster
    # Note that the small basin mergers are not inlcuded
    # in the LGNClusters. Only large basin mergers such that
    # small basin mergers results in X chosen basins.
    LeidenClusters=NodeClustering(communities=basins.Rcommunities,
                           graph=basins.G,
                           method_name="consensus_ledien",
                           method_parameters={
                               "resolution_parameter": resolution,
                               "runs": ensembleSize,
                               "distance_threshold": 0.3}
                          )

    LGNClusters=NodeClustering(communities=basins.communitiesFinal,
                           graph=basins.G,
                           method_name="consensus_ledien",
                           method_parameters={
                               "resolution_parameter": resolution,
                               "runs": ensembleSize,
                               "distance_threshold": 0.3}
                          )


    # Calculate community detection metrics
    for cluster, method in zip([LeidenClusters, LGNClusters], ["LeidenClusters", "LGNClusters"]):
        newman_girvan_modularity = evaluation.newman_girvan_modularity(basins.G, cluster)
        internal_edge_density = evaluation.internal_edge_density(basins.G, cluster)
        erdos_renyi_modularity= evaluation.erdos_renyi_modularity(basins.G, cluster)
        modularity_density    = evaluation.modularity_density(basins.G, cluster)
        avg_embeddedness      = evaluation.avg_embeddedness(basins.G, cluster)
        conductance           = evaluation.conductance(basins.G, cluster)
        surprise              = evaluation.surprise(basins.G, cluster)

        # Add community evaluation metrics to output
        readmetxt += "\n\nCommunity evaluation metrics ({}):\n".format(method);
        readmetxt += "newman_girvan_modularity:\t {}\n".format(newman_girvan_modularity.score)
        readmetxt += "erdos_renyi_modularity:\t\t {}\n".format(erdos_renyi_modularity.score)
        readmetxt += "modularity_density:\t\t {}\n".format(modularity_density.score)
        readmetxt += "internal_edge_density:\t\t {} +- {} (std)\n".format(internal_edge_density.score, internal_edge_density.std)
        readmetxt += "avg_embeddedness:\t\t {} +- {} (std)\n".format(avg_embeddedness.score, avg_embeddedness.std)
        readmetxt += "conductance:\t\t\t {} +- {} (std)\n".format(conductance.score, conductance.std)
        readmetxt += "surprise:\t\t\t {}\n".format(surprise.score)
        
    
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)


# F07: (Leiden Ensemble - Reduction, Girvan-Newman, and Post-Processing On Etopo Bathymetry)

Makes two subplots:
- Etopo bathymetry map
- Leiden-Girvan-Newman detected communities with merged small communities

Note that mergeSmallBasins is now always True, such that we can see the effect of Leiden-Girvan-Newman mergers plus the small basin mergers.


In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset

# Create nodeclustering object
from cdlib import evaluation
from cdlib import NodeClustering

#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden-Girvan-Newman"

# Define basin merging criteria
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [0.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    print(resolution, ensembleSize, minBasinCnt)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage,
                       "njobs":4}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/CompositeMethod_Leiden-GirvanNewman-PP')
    print("Storing images in {}".format(fldName))
    
    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));


    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_1.0deg.nc".format(body),
                             body=body);


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)


    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)


    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)

    #########################
    ### Plot input fields ###
    #########################
    plotRange = [0, 6000];
    N = 1000
    blues_cm = mpl.colormaps['Blues'].resampled(N)
    #plotRange = [ave-1*std, ave+1*std];
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "boundaryColor":'k',
                                 "boundaryLinewidth":1.5,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    

# F07: Node locations for 1) Leiden Ensemble - Reduction and 2) Leiden Ensemble - Reduction and Girvan-Newman On Etopo Bathymetry

Makes two subplots:
- Node locations for initial global graph inputted into the Leiden Ensemble reduction step
- Node locations for the reduced Leiden Ensemble community structure

In [ ]:
# Create node plot:
#     1) Base 1 degree resolution
#     2) Louvain community nodes (i.e., input to Girvan-Newman)
#
####################################################
### Define functions for plotting graph networks ###
####################################################
def split_antimeridian_line(lon1, lat1, lon2, lat2):
    """
    Split a line crossing the antimeridian into two segments,
    one in the eastern hemisphere and one in the western.

    Returns:
        western_segment: ((lon_a, lat_a), (lon_b, lat_b)) or None
        eastern_segment: ((lon_c, lat_c), (lon_d, lat_d)) or None
    """
    # Normalize longitudes to [-180, 180]
    lon1 = (lon1 + 180) % 360 - 180
    lon2 = (lon2 + 180) % 360 - 180

    if abs(lon2 - lon1) <= 180:
        # No crossing: return the full segment and None for the other
        if lon1 < 0:
            return ((lon1, lat1), (lon2, lat2)), None
        else:
            return None, ((lon1, lat1), (lon2, lat2))

    # Antimeridian crossing
    # Determine direction of crossing
    if lon1 > lon2:
        lon2 += 360
    else:
        lon1 += 360

    # Interpolation factor for crossing at 180°
    frac = (180 - lon1) / (lon2 - lon1)
    lat_mid = lat1 + frac * (lat2 - lat1)

    # First segment: lon1 -> 180°
    western_segment = ((lon1, lat1), (180, lat_mid))
    # Second segment: -180° -> lon2
    eastern_segment = ((180, lat_mid), (lon2, lat2))

    return western_segment, eastern_segment

def plotNodes(G, latitude, longitude,
              drawEdges=False, showSideEdges=True,
              markersize =.3,
              fidName='test.png', savePNG=False, saveSVG=False):
    
    # Imports
    import cartopy.crs as ccrs # type: ignore
    # Plot the network on a geographic map
    fig, ax = plt.subplots(figsize=(10, 5), subplot_kw={'projection': ccrs.Mollweide()})
    ax.set_global()

    # Draw the edges (connections)
    if drawEdges:
        for edge in G.edges(data=True):
            node1, node2, weight = edge
            lon1, lat1 = longitude[node1], latitude[node1]
            lon2, lat2 = longitude[node2], latitude[node2]
            cutoff = 90;
            if ((lon2<-cutoff) & (lon1>cutoff)) | ((lon1<-cutoff) & (lon2>cutoff)):
                # Node connection passes through antimeridian
                if showSideEdges:
                    #lats, lons = split_antimeridian([lon1, lon2], [lat1, lat2])
                    segment1, segment2 = split_antimeridian_line(lon1, lat1, lon2, lat2)
                    ax.plot([segment1[0][0],segment1[1][0]], [segment1[0][1],segment1[1][1]],
                            'k-', linewidth=1, transform=ccrs.PlateCarree())
                    ax.plot([segment2[0][0],segment2[1][0]], [segment2[0][1],segment2[1][1]],
                            'k-', linewidth=1, transform=ccrs.PlateCarree())
                pass
            else:
                # Node connection does not pass through antimeridian
                ax.plot([lon1, lon2], [lat1, lat2], 'k-', linewidth=1, transform=ccrs.PlateCarree())
            
            
    # Draw the nodes (points) on the map
    ax.plot(longitude, latitude, 'ro', markersize=markersize, transform=ccrs.PlateCarree())  # longitude, latitude

    # Add coastlines and gridlines
    ax.coastlines()
    ax.gridlines()

    #plt.title("Geographic Network of points")
    
    # Save figure
    if savePNG:
        plt.savefig(fidName, dpi=600, transparent=True)
    if saveSVG:
        plt.savefig(fidName.replace(".png", ".svg"))

        
######################################
# Add details of community reduction #
######################################
        
# Make folder to hold figure results
fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/CompositeMethod_Leiden-GirvanNewman-PP-Nodes')
print("Storing images in {}".format(fldName))

# Short readme text to write to folder with images
readmetxt = "Bathymetry used to calculate basin boundaries.";
readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    
    
with open(fldName+"/readme.txt", "w") as text_file:
    text_file.write(readmetxt)

###############################
# 1) Base 1 degree resolution #
###############################
# Define node locations (latitude/longitude)
baseNodesLat = np.array([],dtype=float);
baseNodesLon = np.array([],dtype=float);
for x in basins.G:
    baseNodesLat = np.append(baseNodesLat, basins.G.nodes[x]['pos'][0])
    baseNodesLon = np.append(baseNodesLon, basins.G.nodes[x]['pos'][1])


plotNodes(basins.G, baseNodesLat, baseNodesLon,
          drawEdges=False, showSideEdges=False,
          fidName=fldName+'/BaseNodes.png',
          savePNG=True)


#############################################################
# 2) Louvain community nodes (i.e., input to Girvan-Newman) #
#############################################################
import copy as cp
LNodesLat = np.array([],dtype=float);
LNodesLon = np.array([],dtype=float);

for x in basins.Gnew:
    # Set nodes idx that are compiled into new nodes for
    # girvan-newman algorithm
    nodesInGnew = np.array(list(basins.Rcommunities[x]), dtype=np.int32)
    # Set average latitude and longitude of nodes in louvain community
    cutoff = 90;
    if (np.min(baseNodesLon[nodesInGnew])<-cutoff) & (np.max(baseNodesLon[nodesInGnew])>cutoff):
        # At the -180,180 boundary find the mean latitude/longitude
        # A mean is fine since nodes represent equal area quadrangles.
        Lons = cp.deepcopy(baseNodesLon[nodesInGnew]);
        Lons[Lons<0] += 360;
        aveLon = np.mean(Lons)
        if aveLon>180:
            aveLon -=360;
    else:
        aveLon = np.mean(baseNodesLon[nodesInGnew])
    
    LNodesLat = np.append(LNodesLat, np.mean(baseNodesLat[nodesInGnew]) )
    LNodesLon = np.append(LNodesLon, aveLon )


#########################
### Plot input fields ###
#########################
plotNodes(basins.Gnew, LNodesLat, LNodesLon,
          drawEdges=True, showSideEdges=True,
          markersize =3,
          fidName=fldName+'/LeidenCommunityNodes.png',
          savePNG=True)


# F05: Weight Construction Using the DQT-CDF Method

This method is described in the text

In [ ]:
########################################################
#### Define functions to show weight scheme process ####
########################################################
# Scheme processes
# 1. Create distribution of absolute value differences between adjacent node  
# 2. Mirror distribition about 0 difference value (D)
# 3. IQR filter to remove outliers (D_{filter})
# 4. Normally distribute the using a data quantile transformation (D_{QT}) 
# 5. Construct shorten and shifted distribution and cumulative density function
# 6. Plot everything to construct manuscript figure

def createDataTreatmentFigures(basins, field,
                               RawDiff=True, IQRFilter=True,  QT=True, QT2=True, QT_CDF=True, 
                               compiledFig=True,
                               savePNG=False, saveSVG=False, directory=os.getcwd()):
    """
    createDataTreatmentFigures create figures to
    visualize the data treatment process in making
    graph edge weights.
    
    Parameters
    -----------
    RawDiff : BOOLEAN
        Plot a distribution of the raw differences
        between connected nodes within the graph.
    IQRFilter : BOOLEAN
        Plot a distribution of the IQR filtered
        differences between connected nodes within
        the graph.
    QT : BOOLEAN
        Plot a distribution of the IQR filtered
        and QT distributed differences between
        connected nodes within the graph.
    QT2 : BOOLEAN
        Similar to QT option.
    QT_CDF : BOOLEAN
        Plot a distribution of the QT distributed
        differences between connected nodes within
        the graph, with a overlay of the CDF function
        used to determine node edge weights.
    compiledFig : BOOLEAN
        2x2 subplots of A) raw difference, B) IQR filtered
        difference, C) DQT-IQR, D) Weight distributions with
        CDF lines.
    savePNG : BOOLEAN
        An option to plot PNGs.
    saveSVG : BOOLEAN
        An option to plot SVGs.
    directory : STRING
        An output directory for the saved figures.
    
    """
    # Set a directory / general name for figure naming convention.
    fidName = basins.Fields[field]['parameterName']
    
    ###################################
    ############# Imports #############
    ###################################
    from scipy.stats import norm
    from sklearn.preprocessing import QuantileTransformer
    
    
    ###################################
    ########## Create figures #########
    ###################################
    if RawDiff:
        # Define mirrored difference data
        xValues = np.append(basins.Fields[field]['dataEdgeDiff'], -basins.Fields[field]['dataEdgeDiff'])
        bins   = np.linspace(np.min(xValues), np.max(xValues), 20);
        
        # Plot subplot
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6.4, 4.8/2));

        # Compare original vs QT 
        hist = plt.hist(xValues, alpha=.5, color='crimson', bins=bins, label='Raw Difference', density=True)
        plt.vlines(x=[np.min(xValues), np.max(xValues)],
                   ymin=0, ymax=np.max(hist[0]), colors='r', alpha=.2,
                   label='Max difference')
        
        # Formatting
        plt.ylabel("UnNormalized Distribution")

        # Plot formatting
        plt.legend();
        plt.xlabel("Difference in Node Property");
        axes.spines['top'].set_visible(False)
        axes.spines['right'].set_visible(False)
        
        # Save figure
        if savePNG:
            plt.savefig(directory+"RawDiff_{}.png".format(fidName), dpi=600, transparent=True)
        if saveSVG:
            plt.savefig(directory+"RawDiff_{}.svg".format(fidName))
        
        
    if IQRFilter:
        # Define mirrored difference data
        xValues = np.append(basins.Fields[field]['dataEdgeDiffIQRFiltered'], -basins.Fields[field]['dataEdgeDiffIQRFiltered'])
        bins   = np.linspace(np.min(xValues), np.max(xValues), 20);
        
        # Plot subplot
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6.4, 4.8/2));

        # Compare original vs QT 
        hist = plt.hist(xValues, alpha=.5, color='crimson', bins=bins, label='IQR Filtered', density=True)
        plt.vlines(x=[np.min(xValues), np.max(xValues)],
                   ymin=0, ymax=np.max(hist[0]), colors='r', alpha=.2,
                   label='Max difference')
        
        # Formatting
        plt.ylabel("UnNormalized Distribution")

        # Plot formatting
        plt.legend();
        plt.xlabel("Difference in Node Property");
        axes.spines['top'].set_visible(False)
        axes.spines['right'].set_visible(False)
        
        # Save figure
        if savePNG:
            plt.savefig(directory+"IQRFilter_{}.png".format(fidName), dpi=600, transparent=True)
        if saveSVG:
            plt.savefig(directory+"IQRFilter_{}.svg".format(fidName))

        
    if QT:
        xValues = np.append(basins.Fields[field]['dataEdgeDiffIQRFiltered'], -basins.Fields[field]['dataEdgeDiffIQRFiltered'])
        basins.Fields[field]['weightMethodPara']['qt'] = \
        QuantileTransformer(n_quantiles=1000,
                            random_state=0,
                            output_distribution='normal')
        qtDiss  = basins.Fields[field]['weightMethodPara']['qt'].fit_transform(np.reshape(xValues, (len(xValues),1)))

        # Create a set of equal space values in the data domain
        # These can be plotted on the gaussian domain to see the data stretching
        bins   = np.linspace(np.min(xValues), np.max(xValues), 20);
        binsqt = basins.Fields[field]['weightMethodPara']['qt'].transform(np.reshape(bins, (len(bins),1)))

        # Plot subplot
        fig, axes = plt.subplots(nrows=2, ncols=1);
        # QT distribution
        plt.sca(axes[1])
        plt.hist(qtDiss, alpha=.5, color='b', bins=np.arange(-6, 6, .1), label='IQR Filtered', density=True)
        plt.vlines(x=binsqt, ymin=0, ymax=1.25, colors='r', alpha=.2)

        # Compare original vs QT 
        plt.sca(axes[0])
        hist = plt.hist(xValues, alpha=.5, color='crimson', bins=bins, label='IQR Filtered', density=True)
        plt.vlines(x=bins, ymin=0, ymax=np.max(hist[0]), colors='r', alpha=.2)
        qtxValues = basins.Fields[field]['weightMethodPara']['qt'].inverse_transform(qtDiss)
        plt.hist(qtxValues, alpha=.5, color='b', bins=bins, label='QT to data', density=True);

        plt.ylabel("UnNormalized Distribution")

        # Plot formatting
        plt.legend();
        plt.sca(axes[1])
        plt.ylim([0,1.25])
        plt.ylabel("Normalized Distribution")
        plt.xlabel("Difference in Node Property [{}/std]".format(basins.Fields[field]['parameterUnit']));
        
        axes[0].spines['top'].set_visible(False)
        axes[0].spines['right'].set_visible(False)
        axes[0].spines['bottom'].set_visible(False)
        
        axes[1].spines['top'].set_visible(False)
        axes[1].spines['right'].set_visible(False)

        
        # Save figure
        if savePNG:
            plt.savefig(directory+"QT_{}.png".format(fidName), dpi=600, transparent=True)
        if saveSVG:
            plt.savefig(directory+"QT_{}.svg".format(fidName))
            
    if QT2:
        xValues = np.append(basins.Fields[field]['dataEdgeDiffIQRFiltered'], -basins.Fields[field]['dataEdgeDiffIQRFiltered'])
        basins.Fields[field]['weightMethodPara']['qt'] = \
        QuantileTransformer(n_quantiles=1000,
                            random_state=0,
                            output_distribution='normal')
        qtDiss  = basins.Fields[field]['weightMethodPara']['qt'].fit_transform(np.reshape(xValues, (len(xValues),1)))

        # Create a set of equal space values in the data domain
        # These can be plotted on the gaussian domain to see the data stretching
        bins   = np.linspace(np.min(xValues), np.max(xValues), 20);
        binsqt = basins.Fields[field]['weightMethodPara']['qt'].transform(np.reshape(bins, (len(bins),1)))

        # Plot subplot
        fig, axes = plt.subplots(nrows=2, ncols=1);
        # QT distribution
        plt.sca(axes[1])
        plt.hist(qtDiss, alpha=.5, color='b', bins=np.arange(-6, 6, .1), label='DQT-IQR', density=True)
        plt.vlines(x=binsqt, ymin=0, ymax=1.25, colors='r', alpha=.2)

        # Compare original vs QT 
        plt.sca(axes[0])
        hist = plt.hist(xValues, alpha=.5, color='crimson', bins=bins, label='IQR Filtered', density=True)
        plt.vlines(x=bins, ymin=0, ymax=np.max(hist[0]), colors='r', alpha=.2)
        qtxValues = basins.Fields[field]['weightMethodPara']['qt'].inverse_transform(qtDiss)
        #plt.hist(qtxValues, alpha=.5, color='b', bins=bins, label='QT to data', density=True);

        plt.ylabel("UnNormalized Distribution")

        # Plot formatting
        plt.legend();
        plt.sca(axes[1])
        plt.legend();
        plt.ylim([0,1.25])
        plt.ylabel("Normalized Distribution")
        plt.xlabel("Difference in Node Property [{}/std]".format(basins.Fields[field]['parameterUnit']));
        
        axes[0].spines['top'].set_visible(False)
        axes[0].spines['right'].set_visible(False)
        axes[0].spines['bottom'].set_visible(False)
        
        axes[1].spines['top'].set_visible(False)
        axes[1].spines['right'].set_visible(False)

        
        # Save figure
        if savePNG:
            plt.savefig(directory+"QT_{}.png".format(fidName), dpi=600, transparent=True)
        if saveSVG:
            plt.savefig(directory+"QT_{}.svg".format(fidName))
        
        
    if QT_CDF:
        # Set data as IQRFiltered that is mirrored on the at dataEdgeDiffIQRFiltered=0
        # (no difference in adjacent node values)
        xValuesFiltered = np.append(basins.Fields[field]['dataEdgeDiffIQRFiltered'],
                                    -basins.Fields[field]['dataEdgeDiffIQRFiltered'])

        # Create QT object and fit
        qt = QuantileTransformer(n_quantiles=1000, random_state=0,  output_distribution='normal')
        x  = qt.fit_transform(np.reshape(xValuesFiltered, (len(xValuesFiltered),1)))

        # Use the QT to convert the xValuesFiltered difference data into z-score space. 
        xFiltered  = qt.fit_transform(np.reshape(xValuesFiltered, (len(xValuesFiltered),1)))

        # Define the cumulative density function in the z-score space. 
        stretchFactor = 3;
        qtDissSTD = 1
        cdfCenter  = qtDissSTD*1
        cdfStretch = qtDissSTD/stretchFactor
        normCDF = 1-1*norm.cdf(xFiltered, loc=0, scale=cdfStretch)
        normCDF2 = 1-1*norm.cdf(xFiltered, loc=0, scale=1)

        distribution = norm.ppf(normCDF.T[0], loc=cdfCenter, scale=cdfStretch/stretchFactor)


        ################################
        ############# Plot #############
        ################################
        # Plot subplot
        fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6.4, 4.8/2));


        
        # QT distribution
        plt.sca(axes)

        plt.hist(xFiltered, alpha=.5, color='b', bins=np.arange(-6, 6, .1), label='DQT-IQR', density=True);
        sorted_indices = np.argsort(normCDF2.T[0])
        plt.plot(xFiltered[sorted_indices], normCDF2.T[0][sorted_indices], 'b-',  alpha=.5, label='1-CDF(DQT-IQR)');

        minWeight = 0.01;
        plt.hist(distribution, alpha=.5, color='r', bins=np.arange(-6, 6, .1), label='Shortened/Shifted', density=True);
        sorted_indices = np.argsort( (normCDF.T[0]+minWeight)/(1+minWeight) )
        plt.plot( (xFiltered+cdfCenter)[sorted_indices], ((normCDF.T[0]+minWeight)/(1+minWeight))[sorted_indices], 'r-',  alpha=.5,
                 label='S = 1-CDF\nshorten = $\sigma/{}$\nshift = $\sigma$'.format(stretchFactor));

        # Formatting
        plt.legend();
        plt.xlabel(r"Difference in Node Property $[\sigma]$");
        plt.ylabel("Normalized weight")
        plt.ylim([0,1.25])
        axes.spines['top'].set_visible(False)
        axes.spines['right'].set_visible(False)

        # Save figure
        if savePNG:
            plt.savefig(directory+"QT_CDF_{}.png".format(fidName), dpi=600, transparent=True)
        if saveSVG:
            plt.savefig(directory+"QT_CDF_{}.svg".format(fidName))
            
        
    if compiledFig:
        # 2x2 Subplots
        # A) raw difference
        # B) IQR filtered difference
        # C) DQT-IQR
        # D) Weight distributions with CDF lines.
        
        #############################
        ####### Create figure #######
        #############################
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(8,5));
        
        
        #########################
        ####### Subplot 1 #######
        #########################
        # Set axis
        plt.sca(axes[0][0])
        
        # Define mirrored difference data
        rawDiff = np.append(basins.Fields[field]['dataEdgeDiff'], -basins.Fields[field]['dataEdgeDiff'])
        bins   = np.linspace(np.min(rawDiff), np.max(rawDiff), 20);
        
        # Compare original vs QT 
        hist = plt.hist(rawDiff, alpha=.5, color='k', bins=bins, label='Raw Difference', density=True)
        plt.vlines(x=[np.min(rawDiff), np.max(rawDiff)],
                   ymin=0, ymax=np.max(hist[0]), colors='r', alpha=.5,
                   label='Max difference')
        
        # Formatting
        plt.ylabel("UnNormalized\nDistribution")

        # Plot formatting
        plt.legend();
        plt.xlabel("Difference in Node Property [m]");
        axes[0][0].spines['top'].set_visible(False)
        axes[0][0].spines['right'].set_visible(False)
        
        plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        
        #########################
        #### For subplot 2/3 ####
        #########################
        IQRFiltered = np.append(basins.Fields[field]['dataEdgeDiffIQRFiltered'], -basins.Fields[field]['dataEdgeDiffIQRFiltered'])
        basins.Fields[field]['weightMethodPara']['qt'] = \
        QuantileTransformer(n_quantiles=1000,
                            random_state=0,
                            output_distribution='normal')
        qtDiss  = basins.Fields[field]['weightMethodPara']['qt'].fit_transform(np.reshape(IQRFiltered, (len(IQRFiltered),1)))

        # Create a set of equal space values in the data domain
        # These can be plotted on the gaussian domain to see the data stretching
        bins   = np.linspace(np.min(IQRFiltered), np.max(IQRFiltered), 20);
        binsqt = basins.Fields[field]['weightMethodPara']['qt'].transform(np.reshape(bins, (len(bins),1)))

        #########################
        ####### Subplot 2 #######
        #########################
        # Set axis
        plt.sca(axes[0][1])
        
        # Define mirrored difference data
        IQRFiltered = np.append(basins.Fields[field]['dataEdgeDiffIQRFiltered'], -basins.Fields[field]['dataEdgeDiffIQRFiltered'])
        bins   = np.linspace(np.min(IQRFiltered), np.max(IQRFiltered), 20);
        
        # Plot subplot
        #fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(6.4, 4.8/2));

        # Compare original vs QT 
        hist = plt.hist(IQRFiltered, alpha=.5, color='orange', bins=bins, label='IQR Filtered', density=True)
        plt.vlines(x=bins, ymin=0, ymax=np.max(hist[0]), colors='k', alpha=.5)
        qtxValues = basins.Fields[field]['weightMethodPara']['qt'].inverse_transform(qtDiss)
        
        # Formatting
        #axes[0][1].yaxis.set_label_position("right")
        #plt.ylabel("UnNormalized Distribution")

        # Plot formatting
        plt.legend();
        plt.xlabel("Difference in Node Property [m]");
        axes[0][1].spines['top'].set_visible(False)
        axes[0][1].spines['left'].set_visible(False)
        axes[0][1].spines['right'].set_visible(False)
        
        axes[0][1].set_yticks([])
        
        # Set subplot 1 ylim equal to subplot 2 ylim
        ymin, ymax = axes[0][1].get_ylim()
        axes[0][0].set_ylim(ymin=ymin, ymax=ymax)
        
        #########################
        ####### Subplot 3 #######
        #########################
        # Set axis
        plt.sca(axes[1][0])
        
        # QT distribution
        plt.hist(qtDiss, alpha=.5, color='b', bins=np.arange(-6, 6, .1), label='DQT-IQR', density=True)
        plt.vlines(x=binsqt, ymin=0, ymax=1.25, colors='k', alpha=.5)

        # Plot formatting
        #plt.legend();
        plt.ylim([0,1.25])
        plt.ylabel("Normalized Distribution")
        plt.xlabel(r"Difference in Node Property $[\sigma]$");
        
        axes[1][0].spines['top'].set_visible(False)
        axes[1][0].spines['right'].set_visible(False)
        
                
        #########################
        ####### Subplot 4 #######
        #########################
        # Set axis
        plt.sca(axes[1][1])
        
        
        # Set data as IQRFiltered that is mirrored on the at dataEdgeDiffIQRFiltered=0
        # (no difference in adjacent node values)
        IQRFiltered = np.append(basins.Fields[field]['dataEdgeDiffIQRFiltered'],
                                -basins.Fields[field]['dataEdgeDiffIQRFiltered'])

        # Create QT object and fit
        qt = QuantileTransformer(n_quantiles=1000, random_state=0,  output_distribution='normal')
        x  = qt.fit_transform(np.reshape(IQRFiltered, (len(IQRFiltered),1)))

        # Use the QT to convert the xValuesFiltered difference data into z-score space. 
        xFiltered  = qt.fit_transform(np.reshape(IQRFiltered, (len(IQRFiltered),1)))

        # Define the cumulative density function in the z-score space. 
        stretchFactor = 3;
        qtDissSTD = 1
        cdfCenter  = qtDissSTD*1
        cdfStretch = qtDissSTD/stretchFactor
        normCDF = 1-1*norm.cdf(xFiltered, loc=0, scale=cdfStretch)
        normCDF2 = 1-1*norm.cdf(xFiltered, loc=0, scale=1)

        distribution = norm.ppf(normCDF.T[0], loc=cdfCenter, scale=cdfStretch/stretchFactor)
        
        # Plot
        plt.hist(xFiltered, alpha=.5, color='b', bins=np.arange(-6, 6, .1), label='DQT-IQR', density=True);
        sorted_indices = np.argsort(normCDF2.T[0])
        plt.plot(xFiltered[sorted_indices], normCDF2.T[0][sorted_indices], 'b-',  alpha=.5, label='1-CDF(DQT-IQR)');

        minWeight = 0.01;
        plt.hist(distribution, alpha=.5, color='r', bins=np.arange(-6, 6, .1), label='Shortened/Shifted', density=True);
        sorted_indices = np.argsort( (normCDF.T[0]+minWeight)/(1+minWeight) )
        plt.plot( (xFiltered+cdfCenter)[sorted_indices], ((normCDF.T[0]+minWeight)/(1+minWeight))[sorted_indices], 'r-',  alpha=.5,
                 label='S = 1-CDF\nshorten = $\sigma/{}$\nshift = $\sigma$'.format(stretchFactor));

        # Formatting
        plt.legend();
        plt.xlabel(r"Difference in Node Property $[\sigma]$");
        #plt.ylabel("Normalized weight")
        plt.ylim([0,1.25])
        axes[1][1].spines['top'].set_visible(False)
        axes[1][1].spines['right'].set_visible(False)
        axes[1][1].spines['left'].set_visible(False)
        
        axes[1][1].set_yticks([])

        # More formatting
        plt.tight_layout()
        
        # Save figure
        if savePNG:
            plt.savefig(directory+"Compiled_{}.png".format(fidName), dpi=600, transparent=True)
        if saveSVG:
            plt.savefig(directory+"Compiled_{}.svg".format(fidName))


            
#########################################
#### Run functions to create figures ####
#########################################
# Create the directory if it doesn't exist
directory_name = os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/DQT-CDF-WeightingMethod"
os.makedirs(directory_name, exist_ok=True)

# Create figures using field 1 data (etopo bathymetry)
field = 'Field1'
createDataTreatmentFigures(basins, field,
                           RawDiff=False, IQRFilter=False,  QT=False,
                           QT2=False, QT_CDF=False, compiledFig=True,
                           savePNG=True, saveSVG=True,
                           directory=directory_name+"/")


# F10: Calculate metrics for Changing Gamma Resolution=[0.005-0.05] and Y Ensemble Size [1, 5, 10, 20, 30, 40, 50] For Community Detection Using Leiden Only Method

Subplots (2): metrics of modularity and large basin count given resolution and ensemble size.

Subplots (6): example maps of basins at 10, 30, 50 ensembles for 0.01 and .04 resolution.

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import itertools
import networkx as nx
from netCDF4 import Dataset

# Create nodeclustering object for community evaluation
from cdlib import evaluation
from cdlib import NodeClustering


#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden"

# Define basin merging criteria
mergerPackageName = "None"; # ['threshold'] = [0]

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, .04, 0.045, 0.05];

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [1, 5, 10, 20, 30, 40, 50];



# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts):
    print(resolution, int(ensembleSize), int(minBasinCnt))
    
# Make base directory
directory_name = os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/SensitivityMethodLeidenGammaEnsemble"
os.makedirs(directory_name, exist_ok=True)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts):
    # Change data type to avoid errors
    minBasinCnt = int(minBasinCnt)
    ensembleSize = int(ensembleSize)
    
    # Set detection method parameters
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":False,
                       "mergerPackage":mergerPackage,
                       "njobs":4}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    #    "multiFieldMethod" can be set to min, max, prod, mean.
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01,
                       "multiFieldMethod":"min"}

    # Make folder to hold figure results
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/SensitivityMethodLeidenGammaEnsemble/Run')
    print("Storing images in {}".format(fldName))
    
    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    



    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_1.0deg.nc".format(body),
                             body=body);


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)
    
    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)


    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)



    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)

    #########################
    ### Plot input fields ###
    #########################
    plotRange = [0, 6000];
    N = 1000
    blues_cm = mpl.colormaps['Blues'].resampled(N)
    #plotRange = [ave-1*std, ave+1*std];
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "boundaryColor":'k',
                                 "boundaryLinewidth":1.5,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)


# F06: Multi-field Weight Construction Diagram

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import os

# === Control font sizes here ===
label_fontsize = 18
annotation_fontsize = 14
legend_fontsize = 14

# 1. Create two distributions
shortenFactor = 3
shiftFactor = 1
Field1Dis = np.random.normal(loc=shiftFactor, scale=1/shortenFactor, size=50000)
Field2Dis = np.random.normal(loc=shiftFactor, scale=1/shortenFactor, size=50000)

# 2. Create CDFs
x1 = np.sort(Field1Dis)
y1 = 1 - np.arange(1, len(x1)+1) / len(x1)

x2 = np.sort(Field2Dis)
y2 = 1 - np.arange(1, len(x2)+1) / len(x2)

# 3. Plot CDFs
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
plt.plot(x1, y1, linestyle=(0, (4, 2)),  color='red', label='Field 1')
plt.plot(x2, y2, linestyle=(3, (4, 2)), color='blue', label='Field 2')

# 4. Points
std_point1 = 1
std_point2 = -0.5

mean1, std1 = np.mean(Field1Dis), np.std(Field1Dis)
mean2, std2 = np.mean(Field2Dis), np.std(Field2Dis)

point1_x = mean1 + std_point1 * std1
point2_x = mean2 + std_point2 * std2

point1_y = 1 - np.searchsorted(x1, point1_x) / len(x1)
point2_y = 1 - np.searchsorted(x2, point2_x) / len(x2)

plt.scatter(point1_x, point1_y, color='red', s=80, zorder=5)
plt.scatter(point2_x, point2_y, color='blue', s=80, zorder=5)

weights = np.array([point1_y, point2_y])

plt.hlines(np.min(weights), xmin=0, xmax=2.5, color='black', zorder=4)
plt.hlines(np.max(weights), xmin=0, xmax=2.5, color='black', zorder=4)
plt.hlines(np.mean(weights), xmin=0, xmax=2.5, color='black', zorder=4)
plt.hlines(np.prod(weights), xmin=0, xmax=2.5, color='black', zorder=4)

# Annotate
plt.annotate(f'$S_{1}$', (point1_x, point1_y), textcoords="offset points", xytext=(10,10), ha='left', color='red', fontsize=annotation_fontsize)
plt.annotate(f'$S_{1}$', (point2_x, point2_y), textcoords="offset points", xytext=(10,10), ha='left', color='blue', fontsize=annotation_fontsize)

plt.annotate(f'Minimum ({np.min(weights):0.2f})',  (.1, np.min(weights)), textcoords="offset points", xytext=(10,10), ha='left', fontsize=annotation_fontsize)
plt.annotate(f'Maximum ({np.max(weights):0.2f})',  (.1, np.max(weights)), textcoords="offset points", xytext=(10,10), ha='left', fontsize=annotation_fontsize)
plt.annotate(f'Mean ({np.mean(weights):0.2f})',    (.1, np.mean(weights)), textcoords="offset points", xytext=(10,10), ha='left', fontsize=annotation_fontsize)
plt.annotate(f'Product ({np.prod(weights):0.2f})', (.1, np.prod(weights)), textcoords="offset points", xytext=(10,-15), ha='left', fontsize=annotation_fontsize)

# Formatting
plt.xlim([0, 2])
plt.xticks([0, 0.5, 1, 1.5, 2])
plt.xlabel('Difference in Node Property [$\\sigma$]', fontsize=label_fontsize)
plt.ylabel('1 - Cumulative Density Functions', fontsize=label_fontsize)
plt.legend(fontsize=legend_fontsize)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.grid(False)

# Create directory for saving diagram
directory_name = os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/MultiFieldWeightDiagram"
os.makedirs(directory_name, exist_ok=True)


# Save figure
plt.savefig(directory_name+"/diagram.png", dpi=600, transparent=True)
plt.savefig(directory_name+"/diagram.svg", transparent=True)


# F12: Multi-field Community Detection
#### A-B): Surface Salinity and Temperature Fields (1998)
#### C-D): Quantile input field for Surface Salinity and Temperature Fields 
#### E-F):  Community detection given DQT-CDF (shorten=XX, shifted=YY), Leiden Ensemble Algorithm (gamma=ZZ, ensembleSize=50), and post-process small basin merge with no Girvan-Newman.

#### F12: make surface(>200 m) / intermediate (>200 m, <1000 m) / deep (>1000 m) averaged and time averaged Salinity and Potential Temperature fields.

In [ ]:
###################################
############# Imports #############
###################################
import ExoCcycle as EC
import os
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt


#########################################
############# Define QT/CDF #############
#########################################
# Initialize object (Defaults to using 1994, bottomT data)
dataType = 'so'
options = {"download": False,
           "dataDir":os.getcwd()+"/GLORYS12V1",
           "year":[1994],
           "data": dataType,
           "depthAve": [1000,6000]}

GLORYS12V1 = EC.utils.GLORYS12V1(options)

# Average models all months in 1994
GLORYS12V1.averageModels()
print("Finished Deep Salinity")

# Initialize object (Defaults to using 1994, bottomT data)
dataType = 'thetao'
options = {"download": False,
           "dataDir":os.getcwd()+"/GLORYS12V1",
           "year":[1994],
           "data": dataType,
           "depthAve": [1000,6000]}

GLORYS12V1 = EC.utils.GLORYS12V1(options)

# Average models all months in 1994
GLORYS12V1.averageModels()
print("Finished Deep Temperature")

# Initialize object (Defaults to using 1994, bottomT data)
dataType = 'so'
options = {"download": False,
           "dataDir":os.getcwd()+"/GLORYS12V1",
           "year":[1994],
           "data": dataType,
           "depthAve": [200,1000]}

GLORYS12V1 = EC.utils.GLORYS12V1(options)

# Average models all months in 1994
GLORYS12V1.averageModels()
print("Finished Intermediate Salinity")


# Initialize object (Defaults to using 1994, bottomT data)
dataType = 'thetao'
options = {"download": False,
           "dataDir":os.getcwd()+"/GLORYS12V1",
           "year":[1994],
           "data": dataType,
           "depthAve": [200,1000]}

GLORYS12V1 = EC.utils.GLORYS12V1(options)

# Average models all months in 1994
GLORYS12V1.averageModels()
print("Finished Intermediate Temperature")

# Initialize object (Defaults to using 1994, bottomT data)
dataType = 'so'
options = {"download": False,
           "dataDir":os.getcwd()+"/GLORYS12V1",
           "year":[1994],
           "data": dataType,
           "depthAve": [0,200]}

GLORYS12V1 = EC.utils.GLORYS12V1(options)

# Average models all months in 1994
GLORYS12V1.averageModels()
print("Finished Shallow Salinity")

# Initialize object (Defaults to using 1994, bottomT data)
dataType = 'thetao'
options = {"download": False,
           "dataDir":os.getcwd()+"/GLORYS12V1",
           "year":[1994],
           "data": dataType,
           "depthAve": [0,200]}

GLORYS12V1 = EC.utils.GLORYS12V1(options)

# Average models all months in 1994
GLORYS12V1.averageModels()
print("Finished Shallow Temperature")


#### F12: Create community detection and field plots (A-F)

Assign multiFieldMethod with 'min' and 'mean' to reproduce figure subplots from paper.

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import copy as cp
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import itertools
import networkx as nx
from netCDF4 import Dataset
import cmcrameri.cm as cmc

# Create nodeclustering object for community evaluation
from cdlib import evaluation
from cdlib import NodeClustering

#################################
### User define fields to use ###
#################################
useTemp = True;
useSalinity = True;

#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden"

# Define basin merging criteria
#mergerPackageName = "None"; # ['threshold'] = [0]
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [0.01];

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
#ensembleSizes = [1, 5, 10, 20, 30, 40, 50];
ensembleSizes = [50];

# MultiField merger method
# Can be set to "min", "mean", "max", "prod"
# multiFieldMethod = "mean";
multiFieldMethod = "min";


# Define all depth ranges to calculate community division for
# depthRanges = np.array([[0,200],[200,1000],[1000,6000]]);
depthRanges = np.array([[0,200]]);

# Define fields to detect communities on
subfld = ""

scalarValue1 =  "so_average"
unit1 = "PSU"
scalarValue2 = "thetao_average"
unit2 = "PotTemp"

# Short readme text to write to folder with images
if useTemp & useSalinity:
    usedFieldsStr = "{}/{}".format(unit1,unit2);
    fieldNums = ["Field2", "Field3"]
elif useSalinity:
    usedFieldsStr = "{}".format(unit1);
    fieldNums = ["Field2"]
elif useTemp:
    usedFieldsStr = "{}".format(unit2);
    fieldNums = ["Field3"]



# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts):   
    for i in range(depthRanges.shape[0]):
        # Define ith depth range to evaluate
        depthRange = depthRanges[i,:];
        print(resolution, int(ensembleSize), int(minBasinCnt), multiFieldMethod, depthRange)
    
# Make base directory
directory_name = os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/GLORYS12V1-MultiField"+subfld
os.makedirs(directory_name, exist_ok=True)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts):
    for i in range(depthRanges.shape[0]):
        # Define ith depth range to evaluate
        depthRange = depthRanges[i,:];

        # Change data type to avoid errors
        minBasinCnt = int(minBasinCnt)
        ensembleSize = int(ensembleSize)

        # Set detection method parameters
        detectionMethod = {"method":communityDetectionMethod,
                           "resolution":resolution,
                           "minBasinCnt":minBasinCnt,
                           "ensembleSize":ensembleSize,
                           "minBasinLargerThanSmallMergers":False,
                           "mergerPackage":mergerPackage,
                           "njobs":4}

        # Set the edge weight scheme for node connections
        # Options:
        #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
        #    "multiFieldMethod" can be set to min, max, prod, mean.
        edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                           "shortenFactor": 3,
                           "shiftFactor": 1,
                           "minWeight": 0.01,
                           "multiFieldMethod":multiFieldMethod}

        # Make folder to hold figure results
        fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/GLORYS12V1-MultiField{}/GLORYS12V1_1994_{}_{}'.format(subfld, usedFieldsStr.replace("/", "_"), multiFieldMethod))
        print("Storing images in {}".format(fldName))

        # Short readme text to write to folder with images
        readmetxt = "{} used to calculate basin boundaries.".format(usedFieldsStr);
        readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
        readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
        readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
        readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
        readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
        readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    
        readmetxt += "\nMulti-field merger method {}".format(edgeWeightMethod["multiFieldMethod"])

        #################################################################
        ### Create basin object and set Field for Community detection ###
        #################################################################

        # Create basin object
        body = ["Earth", "Mars", "Venus", "Moon"]
        body = body[0]
        basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                                 filename="{}_resampled_1.0deg.nc".format(body),
                                 body=body);

        # Define the equal area node options s.t. salinity is used for edge connections
        basins.addField(resolution = basins.Fields["Field1"]["resolution"],
                        dataGrid =  os.getcwd()+"/GLORYS12V1/{0}_{1}_{2}m{3}.nc".format(scalarValue1, depthRange[0], depthRange[1], subfld),
                        parameter = "z",
                        parameterUnit = unit1,
                        parameterName = scalarValue1)
        basins.addField(resolution = basins.Fields["Field1"]["resolution"],
                        dataGrid =  os.getcwd()+"/GLORYS12V1/{0}_{1}_{2}m{3}.nc".format(scalarValue2, depthRange[0], depthRange[1], subfld),
                        parameter = "z",
                        parameterUnit = unit2,
                        parameterName = scalarValue2)

        # Assign fields to use in community detection
        if useTemp & useSalinity:
            basins.useFields(fieldList=np.array(["Field2","Field3"]));
        elif useSalinity:
            basins.useFields(fieldList=np.array(["Field2"]));
        elif useTemp:
            basins.useFields(fieldList=np.array(["Field3"]));

        # Show all fields stored in basins object
        basins.getFields(usedFields = False)

        # Show all fields stored in basins object that will be used
        # for community detection.
        basins.getFields(usedFields = True)
        

        #########################################
        ### Run Community Detection Algorithm ###
        #########################################

        # Define basins based on user input boundaries.
        # For the Louvain-Girvan-Newman composite algorithm the variable
        # minBasinCnt refers to the number of basins to maintain that are
        # not completely isolated after running the louvain algorithm.
        basins.defineBasins(detectionMethod = detectionMethod,
                            edgeWeightMethod = edgeWeightMethod,
                            reducedRes={"on":True,"factor":1},
                            read=False,
                            write=True,
                            verbose=False)


        # Merge communities based off criteria
        # Note that etopo is used as a mask for merging small basins and
        # plotting communities.
        basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

        # Convert basinID equal area grid to regular grid
        basins.interp2regularGrid(mask=True)

        # Get small and large basins
        percentThreshold = 0.5;
        basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)

        readmetxt += basinSizeDic["text"]

        # Get community detection metrics
        metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                             resolution=resolution,
                                                             ensembleSize=ensembleSize,
                                                             distance_threshold=0.3)
        readmetxt += ("\n\n" + metricText)
        
        ##################################
        ### Calculate Silhouette field ###
        ##################################
        silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

        # Calculate area weighted average and standard deviation (for plotting)
        areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                                  LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                                  LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
        ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
        aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
        print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
        readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  


        ###########################################
        ### Report community evaluation metrics ###
        ###########################################
        with open(fldName+"/readme.txt", "w") as text_file:
            text_file.write(readmetxt)
            
        #####################################
        ### Plot results of community IDs ###
        #####################################
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal.png",
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                            pltOpts={"valueType": "BasinID divided by {}".format(usedFieldsStr.replace("/", " & ")),
                                     "valueUnits": "-",
                                     "plotTitle":"",
                                     "mesh":True,
                                     "plotZeroContour":False,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)

        ##############################
        ### Plot Silhouette fields ###
        ##############################
        EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,
                                                    1],
                                      "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                      "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                      "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                            pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                     "valueUnits": "{}".format("-"),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "nanSolidPoly":True,
                                     "nanSolidPolyOutline":True,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)

        
        #####################
        ### Plot Salinity ###
        #####################
        if useSalinity:
            # Write temp.nc that only has lat/lon/value
            #basins.simplifyNetCDF(inputPath=basins.EAinputs['dataGrid'],
            #                    outputPath='tempSimp.nc',
            #                    parameter=basins.EAinputs['parameter'])
            # Read netCDF4
            nc = Dataset('tempSimp_{}.nc'.format(scalarValue1))
            XX, YY = np.meshgrid(nc['lon'][:].data, nc['lat'][:].data)

            # Calculate area weighted average and standard deviation (for plotting)
            areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=np.diff(nc['lon'][:].data)[0],
                                                                                                      LonStEd = [-180,180],
                                                                                                      LatStEd = [-80-np.diff(nc['lon'][:].data)[0],90])
            ave, std = EC.utils.weightedAvgAndStd(nc['z'][:].data, areaWeights)

            # Plot PSU at depth interval
            EC.plotHelper.plotGlobal(YY, XX, nc['z'][:].data,
                                outputDir = os.getcwd()+"/"+fldName,
                                fidName = "plotGlobal_{0}_{1}m_{2}.png".format(depthRange[0], depthRange[1], unit1),
                                cmapOpts={"cmap":"jet",
                                          "cbar-title":"cbar-title",
                                          "cbar-range":[ave-1*std,
                                                        ave+1*std]},
                                pltOpts={"valueType": "{2} {0}-{1} m".format(depthRange[0], depthRange[1], unit1),
                                         "valueUnits": "1e3",
                                         "plotTitle":"",
                                         "plotZeroContour":False,
                                         "transparent":True},
                                savePNG=True,
                                saveSVG=False)
            nc.close()

        ##################################
        ### Plot Potential Temperature ###
        ##################################
        if useTemp:
            # Write temp.nc that only has lat/lon/value
            #basins.simplifyNetCDF(inputPath=basins.EAinputs['dataGrid'],
            #                    outputPath='tempSimp.nc',
            #                    parameter=basins.EAinputs['parameter'])
            # Read netCDF4
            nc = Dataset('tempSimp_{}.nc'.format(scalarValue2))
            XX, YY = np.meshgrid(nc['lon'][:].data, nc['lat'][:].data)

            # Calculate area weighted average and standard deviation (for plotting)
            areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=np.diff(nc['lon'][:].data)[0],
                                                                                                      LonStEd = [-180,180],
                                                                                                      LatStEd = [-80-np.diff(nc['lon'][:].data)[0],90])
            ave, std = EC.utils.weightedAvgAndStd(nc['z'][:].data, areaWeights)

            # Plot PSU at depth interval
            EC.plotHelper.plotGlobal(YY, XX, nc['z'][:].data,
                                outputDir = os.getcwd()+"/"+fldName,
                                fidName = "plotGlobal_{0}_{1}m_{2}.png".format(depthRange[0], depthRange[1], unit2),
                                cmapOpts={"cmap":"jet",
                                          "cbar-title":"cbar-title",
                                          "cbar-range":[ave-1*std,
                                                        ave+1*std]},
                                pltOpts={"valueType": "{2} {0}-{1} m".format(depthRange[0], depthRange[1], unit2),
                                         "valueUnits": "C",
                                         "plotTitle":"",
                                         "plotZeroContour":False,
                                         "transparent":True},
                                savePNG=True,
                                saveSVG=False)
            nc.close()


        ###########################
        ### Plot DQT-CDF Values ###
        ###########################
        for fieldNum in fieldNums:
            #####################################################
            ####### Find average node neighbor difference #######
            #####################################################
            # Iterate over each node
            attrs = None;
            for node in basins.G.nodes:
                # Average node connection difference for each node
                temp = 0; cnt = 0;
                for conNode in basins.G.neighbors(node):
                    temp+= np.abs( basins.G.nodes[conNode][fieldNum]-basins.G.nodes[node][fieldNum] )
                    cnt+=1;
                try:
                    diffAve = temp/cnt; 
                except:
                    # If node has no connections. This rarely happens.
                    #print(temp, cnt, basins.G.neighbors(node) )
                    diffAve = 0
                # Collect average node neighbor difference property
                if attrs == None:
                    attrs = {node: {"diffAve": diffAve}};
                else:
                    attrs[node] = {"diffAve": diffAve};

            # Assign average node neighbor difference node property to graph
            G = nx.set_node_attributes(basins.G, attrs)

            # List values
            diffAve_values = list(basins.G.nodes(data="diffAve"))


            #####################################################
            ################ Interpolate to grid ################
            #####################################################
            def interp2regularGrid(basins, dataIrregular=None, mask=True):
                """
                interp2regularGrid method is used to interpolate data to
                a regular grid given an input of irregular spaced data.

                Parameters
                -----------
                dataIrregular : NUMPY ARRAY
                    3XN numpy array with columns of longitude, latitude, magnitude.
                    The default is None. This will make the function define the 
                    dataIrregular variable with basinIDs.
                mask : STRING
                    The path to a netCDF4 file that can be used to mask the result
                    of interpolation. The default is None.

                Returns
                --------
                array : NUMPY ARRAY
                    A 2nxn array that hold node properties for each corresponding
                    entry in basins.lat and basins.lon. 

                """

                import copy as cp

                # Get basin IDs from network object.
                tmpValuesID  = nx.get_node_attributes(basins.G, "diffAve");
                tmpValuesPos = nx.get_node_attributes(basins.G, "pos");

                # Define an array to hold longitude, latitude, and basinID
                dataIrregular = np.zeros((len(tmpValuesPos), 3))

                # Iterate over all nodes so each node's longitude, latitude,
                # and basinID can be added to the dataIrregular array.
                for i in tmpValuesID:
                    dataIrregular[i,:] = np.array([tmpValuesPos[i][1], tmpValuesPos[i][0], tmpValuesID[i]])

                # Define an array 2nxn to hold the basin IDs for the regular grid
                # on the surface of the a sphere (planet). 
                array = cp.deepcopy(basins.lat)

                # Define a mapping function that maps node indecies on a irregular grid
                # to those on the regular grid. This will speed up calculations if this
                # function is called more than once.

                # Iterate over all latitude and longitudes of the input grid.
                for i in range(len(basins.lat[:,0])):
                    for j in range(len(basins.lat[0,:])):
                        # Find the distances from each regular grid point (i,j) to all
                        # irregular grid points.
                        x = EC.utils.haversine_distance(lat2= dataIrregular[:,1], lat1= basins.lat[i,j],
                                                lon2= dataIrregular[:,0], lon1= basins.lon[i,j],
                                                radius=1)

                        # Assign the nearest basin ID to element (i,j) 
                        array[i,j] = dataIrregular[np.argwhere(np.nanmin(x) == x)[0][0], 2]

                ## Apply the mask
                if mask:
                    array[np.isnan(basins.maskValue)] = np.nan


                return array;

            # Interpolate from equal area grid to regular spaced latitude/longitude
            diffAveGrd = interp2regularGrid(basins, mask=True)

            #####################################################
            ####################### Plot ########################
            #####################################################
            # Transform using the QT
            DQT_zscore = cp.deepcopy(diffAveGrd);
            shape = np.shape(DQT_zscore)
            DQT_zscore = np.reshape(DQT_zscore, (np.size(DQT_zscore), 1) )
            DQT_zscoreNonNans = cp.deepcopy( DQT_zscore[~np.isnan(DQT_zscore)] )
            DQT_zscoreNonNans =  basins.Fields[fieldNum]['weightMethodPara']['qt'].transform( np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans), 1) ) )             
            DQT_zscore[~np.isnan(DQT_zscore)] = np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans),) )
            DQT_zscore = np.reshape(DQT_zscore, shape)


            # Plot using ExoCcycle plotGlobal function
            EC.plotHelper.plotGlobal(basins.lat, basins.lon, DQT_zscore,
                                outputDir = os.getcwd()+"/"+fldName,
                                fidName = "plotGlobal_DQT_{}.png".format(basins.Fields[fieldNum]['parameterUnit']),
                                cmapOpts={"cmap":cmc.batlow,
                                          "cbar-title":"cbar-title",
                                          "cbar-range":[0,
                                                        2]},
                                pltOpts={"valueType": "DQT",
                                         "valueUnits": "zscore",
                                         "plotTitle":"",
                                         "plotZeroContour":False,
                                         "transparent":True},
                                savePNG=True,
                                saveSVG=False)




# F11: Community Detection throughout water column (Shallow/Intermediate/Deep): Potential temperature or Salinity

Change useTemp, useSalinity, and communityDetectionMethod to produce basin detection at different water column depths, using either the temperature or salinity field with either the Leiden or Louvain detection method.

Assign communityDetectionMethod to 'Leiden' and 'Louvain' to reproduce paper figure subplots.

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import copy as cp
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import itertools
import networkx as nx
from netCDF4 import Dataset
import cmcrameri.cm as cmc

# Create nodeclustering object for community evaluation
from cdlib import evaluation
from cdlib import NodeClustering

#################################
### User define fields to use ###
#################################
useTemp = True;
useSalinity = False;
# useTemp = False;
# useSalinity = True;


#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
# communityDetectionMethod = "Leiden"
communityDetectionMethod = "Louvain"


# Define basin merging criteria
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [0.01];

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Define all depth ranges to calculate community division for
depthRanges = np.array([[0,200],[200,1000],[1000,6000]]);

# Define fields to detect communities on
subfldPrefix = ""

scalarValue1 =  "so_average"
unit1 = "PSU"
scalarValue2 = "thetao_average"
unit2 = "PotTemp"

# Short readme text to write to folder with images
if useTemp & useSalinity:
    usedFieldsStr = "{}/{}".format(unit1,unit2);
    fieldNums = ["Field2", "Field3"]
elif useSalinity:
    usedFieldsStr = "{}".format(unit1);
    fieldNums = ["Field2"]
elif useTemp:
    usedFieldsStr = "{}".format(unit2);
    fieldNums = ["Field3"]


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts):   
    for i in range(depthRanges.shape[0]):
        # Define ith depth range to evaluate
        depthRange = depthRanges[i,:];
        print(resolution, int(ensembleSize), int(minBasinCnt), depthRange)
    
# Make base directory
subFolder = "GLORYS12V1-WaterColumn{}/{}-{}".format(subfldPrefix, communityDetectionMethod, usedFieldsStr.replace("/","_"))
directory_name = os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/"+subFolder
os.makedirs(directory_name, exist_ok=True)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts):
    for i in range(depthRanges.shape[0]):
        # Define ith depth range to evaluate
        depthRange = depthRanges[i,:];

        # Change data type to avoid errors
        minBasinCnt = int(minBasinCnt)
        ensembleSize = int(ensembleSize)

        # Set detection method parameters
        detectionMethod = {"method":communityDetectionMethod,
                           "resolution":resolution,
                           "minBasinCnt":minBasinCnt,
                           "ensembleSize":ensembleSize,
                           "minBasinLargerThanSmallMergers":False,
                           "mergerPackage":mergerPackage}

        # Set the edge weight scheme for node connections
        # Options:
        #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
        #    "multiFieldMethod" can be set to min, max, prod, mean.
        edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                           "shortenFactor": 3,
                           "shiftFactor": 1,
                           "minWeight": 0.01,
                           "multiFieldMethod":"max"}

        # Make folder to hold figure results
        !mkdir -p figures/GMD_Manuscript/CodeOutputs/GLORYS12V1-MultiField
        fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/{0}/GLORYS12V1_1994_{1}'.format(subFolder, usedFieldsStr.replace("/", "_")))
        print("Storing images in {}".format(fldName))

        # Short readme text to write to folder with images
        readmetxt = "{} used to calculate basin boundaries.".format(usedFieldsStr);
        readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
        readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
        readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
        readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
        readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
        readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    


        #################################################################
        ### Create basin object and set Field for Community detection ###
        #################################################################

        # Create basin object
        body = ["Earth", "Mars", "Venus", "Moon"]
        body = body[0]
        basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                                 filename="{}_resampled_1.0deg.nc".format(body),
                                 body=body);

        # Define the equal area node options s.t. salinity is used for edge connections
        basins.addField(resolution = basins.Fields["Field1"]["resolution"],
                        dataGrid =  os.getcwd()+"/GLORYS12V1/{0}_{1}_{2}m{3}.nc".format(scalarValue1, depthRange[0], depthRange[1], subfldPrefix),
                        parameter = "z",
                        parameterUnit = unit1,
                        parameterName = scalarValue1)
        basins.addField(resolution = basins.Fields["Field1"]["resolution"],
                        dataGrid =  os.getcwd()+"/GLORYS12V1/{0}_{1}_{2}m{3}.nc".format(scalarValue2, depthRange[0], depthRange[1], subfldPrefix),
                        parameter = "z",
                        parameterUnit = unit2,
                        parameterName = scalarValue2)

        # Assign fields to use in community detection
        if useTemp & useSalinity:
            basins.useFields(fieldList=np.array(["Field2","Field3"]));
        elif useSalinity:
            basins.useFields(fieldList=np.array(["Field2"]));
        elif useTemp:
            basins.useFields(fieldList=np.array(["Field3"]));

        # Show all fields stored in basins object
        basins.getFields(usedFields = False)

        # Show all fields stored in basins object that will be used
        # for community detection.
        basins.getFields(usedFields = True)


        #########################################
        ### Run Community Detection Algorithm ###
        #########################################

        # Define basins based on user input boundaries.
        # For the Louvain-Girvan-Newman composite algorithm the variable
        # minBasinCnt refers to the number of basins to maintain that are
        # not completely isolated after running the louvain algorithm.
        basins.defineBasins(detectionMethod = detectionMethod,
                            edgeWeightMethod = edgeWeightMethod,
                            reducedRes={"on":True,"factor":1},
                            read=False,
                            write=True,
                            verbose=False)


        # Merge communities based off criteria 
        basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

        # Convert basinID equal area grid to regular grid
        basins.interp2regularGrid(mask=True)


        # Get small and large basins
        percentThreshold = 0.5;
        basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)

        readmetxt += basinSizeDic["text"]

        # Get community detection metrics
        metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                             resolution=resolution,
                                                             ensembleSize=ensembleSize,
                                                             distance_threshold=0.3)
        readmetxt += ("\n\n" + metricText)
        
        ##################################
        ### Calculate Silhouette field ###
        ##################################
        silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

        # Calculate area weighted average and standard deviation (for plotting)
        areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                                  LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                                  LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
        ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
        aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
        print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
        readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  


        ###########################################
        ### Report community evaluation metrics ###
        ###########################################
        with open(fldName+"/readme.txt", "w") as text_file:
            text_file.write(readmetxt)


        #####################################
        ### Plot results of community IDs ###
        #####################################
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal.png",
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                            pltOpts={"valueType": "BasinID divided by {}".format(usedFieldsStr),
                                     "valueUnits": "-",
                                     "plotTitle":"",
                                     "mesh":True,
                                     "plotZeroContour":False,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)


        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_Contour.png",
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                            pltOpts={"valueType": "BasinID divided by {}".format(usedFieldsStr),
                                     "valueUnits": "-",
                                     "plotTitle":"",
                                     "mesh":False,
                                     "coastlines":False,
                                     "nanSolidPoly":True,
                                     "nanSolidPolyOutline":True,
                                     "plotZeroContour":False,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)
        
        ##############################
        ### Plot Silhouette fields ###
        ##############################
        EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,
                                                    1],
                                      "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                      "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                      "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                            pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                     "valueUnits": "{}".format("-"),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "nanSolidPoly":True,
                                     "nanSolidPolyOutline":True,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)


        #####################
        ### Plot Salinity ###
        #####################
        if useSalinity:
            # Read netCDF4
            nc = Dataset('tempSimp_{}.nc'.format(scalarValue1))
            XX, YY = np.meshgrid(nc['lon'][:].data, nc['lat'][:].data)

            # Calculate area weighted average and standard deviation (for plotting)
            areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=np.diff(nc['lon'][:].data)[0],
                                                                                                      LonStEd = [-180,180],
                                                                                                      LatStEd = [-80-np.diff(nc['lon'][:].data)[0],90])
            ave, std = EC.utils.weightedAvgAndStd(nc['z'][:].data, areaWeights)

            # Plot PSU at depth interval
            EC.plotHelper.plotGlobal(YY, XX, nc['z'][:].data,
                                outputDir = os.getcwd()+"/"+fldName,
                                fidName = "plotGlobal_{0}_{1}m_{2}.png".format(depthRange[0], depthRange[1], unit1),
                                cmapOpts={"cmap":"jet",
                                          "cbar-title":"cbar-title",
                                          "cbar-range":[ave-1*std,
                                                        ave+1*std]},
                                pltOpts={"valueType": "{2} {0}-{1} m".format(depthRange[0], depthRange[1], unit1),
                                         "valueUnits": "1e3",
                                         "plotTitle":"",
                                         "plotZeroContour":False,
                                         "transparent":True},
                                savePNG=True,
                                saveSVG=False)
            nc.close()

        ##################################
        ### Plot Potential Temperature ###
        ##################################
        if useTemp:
            # Read netCDF4
            nc = Dataset('tempSimp_{}.nc'.format(scalarValue2))
            XX, YY = np.meshgrid(nc['lon'][:].data, nc['lat'][:].data)

            # Calculate area weighted average and standard deviation (for plotting)
            areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=np.diff(nc['lon'][:].data)[0],
                                                                                                      LonStEd = [-180,180],
                                                                                                      LatStEd = [-80-np.diff(nc['lon'][:].data)[0],90])
            ave, std = EC.utils.weightedAvgAndStd(nc['z'][:].data, areaWeights)

            # Plot PSU at depth interval
            EC.plotHelper.plotGlobal(YY, XX, nc['z'][:].data,
                                outputDir = os.getcwd()+"/"+fldName,
                                fidName = "plotGlobal_{0}_{1}m_{2}.png".format(depthRange[0], depthRange[1], unit2),
                                cmapOpts={"cmap":"jet",
                                          "cbar-title":"cbar-title",
                                          "cbar-range":[ave-1*std,
                                                        ave+1*std]},
                                pltOpts={"valueType": "{2} {0}-{1} m".format(depthRange[0], depthRange[1], unit2),
                                         "valueUnits": "C",
                                         "plotTitle":"",
                                         "plotZeroContour":False,
                                         "transparent":True},
                                savePNG=True,
                                saveSVG=False)
            nc.close()


        ###########################
        ### Plot DQT-CDF Values ###
        ###########################
        for fieldNum in fieldNums:
            #####################################################
            ####### Find average node neighbor difference #######
            #####################################################
            # Iterate over each node
            attrs = None;
            for node in basins.G.nodes:
                # Average node connection difference for each node
                temp = 0; cnt = 0;
                for conNode in basins.G.neighbors(node):
                    temp+= np.abs( basins.G.nodes[conNode][fieldNum]-basins.G.nodes[node][fieldNum] )
                    cnt+=1;
                try:
                    diffAve = temp/cnt; 
                except:
                    # If node has no connections. This rarely happens.
                    #print(temp, cnt, basins.G.neighbors(node) )
                    diffAve = 0
                # Collect average node neighbor difference property
                if attrs == None:
                    attrs = {node: {"diffAve": diffAve}};
                else:
                    attrs[node] = {"diffAve": diffAve};

            # Assign average node neighbor difference node property to graph
            G = nx.set_node_attributes(basins.G, attrs)

            # List values
            diffAve_values = list(basins.G.nodes(data="diffAve"))


            #####################################################
            ################ Interpolate to grid ################
            #####################################################
            def interp2regularGrid(basins, dataIrregular=None, mask=True):
                """
                interp2regularGrid method is used to interpolate data to
                a regular grid given an input of irregular spaced data.

                Parameters
                -----------
                dataIrregular : NUMPY ARRAY
                    3XN numpy array with columns of longitude, latitude, magnitude.
                    The default is None. This will make the function define the 
                    dataIrregular variable with basinIDs.
                mask : STRING
                    The path to a netCDF4 file that can be used to mask the result
                    of interpolation. The default is None.

                Returns
                --------
                array : NUMPY ARRAY
                    A 2nxn array that hold node properties for each corresponding
                    entry in basins.lat and basins.lon. 

                """

                import copy as cp

                # Get basin IDs from network object.
                tmpValuesID  = nx.get_node_attributes(basins.G, "diffAve");
                tmpValuesPos = nx.get_node_attributes(basins.G, "pos");

                # Define an array to hold longitude, latitude, and basinID
                dataIrregular = np.zeros((len(tmpValuesPos), 3))

                # Iterate over all nodes so each node's longitude, latitude,
                # and basinID can be added to the dataIrregular array.
                for i in tmpValuesID:
                    dataIrregular[i,:] = np.array([tmpValuesPos[i][1], tmpValuesPos[i][0], tmpValuesID[i]])

                # Define an array 2nxn to hold the basin IDs for the regular grid
                # on the surface of the a sphere (planet). 
                array = cp.deepcopy(basins.lat)

                # Define a mapping function that maps node indecies on a irregular grid
                # to those on the regular grid. This will speed up calculations if this
                # function is called more than once.

                # Iterate over all latitude and longitudes of the input grid.
                for i in range(len(basins.lat[:,0])):
                    for j in range(len(basins.lat[0,:])):
                        # Find the distances from each regular grid point (i,j) to all
                        # irregular grid points.
                        x = EC.utils.haversine_distance(lat2= dataIrregular[:,1], lat1= basins.lat[i,j],
                                                lon2= dataIrregular[:,0], lon1= basins.lon[i,j],
                                                radius=1)

                        # Assign the nearest basin ID to element (i,j) 
                        array[i,j] = dataIrregular[np.argwhere(np.nanmin(x) == x)[0][0], 2]

                ## Apply the mask
                if mask:
                    array[np.isnan(basins.maskValue)] = np.nan


                return array;

            # Interpolate from equal area grid to regular spaced latitude/longitude
            diffAveGrd = interp2regularGrid(basins, mask=True)

            #####################################################
            ####################### Plot ########################
            #####################################################
            # Transform using the QT
            DQT_zscore = cp.deepcopy(diffAveGrd);
            shape = np.shape(DQT_zscore)
            DQT_zscore = np.reshape(DQT_zscore, (np.size(DQT_zscore), 1) )
            DQT_zscoreNonNans = cp.deepcopy( DQT_zscore[~np.isnan(DQT_zscore)] )
            DQT_zscoreNonNans =  basins.Fields[fieldNum]['weightMethodPara']['qt'].transform( np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans), 1) ) )             
            DQT_zscore[~np.isnan(DQT_zscore)] = np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans),) )
            DQT_zscore = np.reshape(DQT_zscore, shape)


            # Plot using ExoCcycle plotGlobal function
            EC.plotHelper.plotGlobal(basins.lat, basins.lon, DQT_zscore,
                                outputDir = os.getcwd()+"/"+fldName,
                                fidName = "plotGlobal_DQT_{}.png".format(basins.Fields[fieldNum]['parameterUnit']),
                                cmapOpts={"cmap":cmc.batlow,
                                          "cbar-title":"cbar-title",
                                          "cbar-range":[0,
                                                        2]},
                                pltOpts={"valueType": "DQT",
                                         "valueUnits": "zscore",
                                         "plotTitle":"",
                                         "plotZeroContour":False,
                                         "transparent":True},
                                savePNG=True,
                                saveSVG=False)
            
        # Plot using ExoCcycle plotGlobal function
        # Same as above except plots the weight values
        # excluding the distance dependence between nodes.
        # However, this should be negligible.
        def complementCDF(diff,
                          transformer,
                          std,
                          shortenFactor = edgeWeightMethod['shortenFactor'],
                          shiftFactor = edgeWeightMethod['shiftFactor'],
                          minWeight = edgeWeightMethod['minWeight']):
            """
            complementCDF is function used to calculate the weight of a node
            pair connection given the following inputs.

            Parameters
            -----------
            diff : FLOAT
                A node pair difference value from field of data.
            transformer : OBJECT
                Quantile transformation that is used to convert diff
                input into z-score value.
            std : FLAOT
                Standard devitaion of total field data input after
                being quantile transformed (i.e., this value should
                be ~0). 
            shortenFactor : FLOAT
                Factor to shorten CDF distribution by.
            shiftFactor : FLOAT
                Factor to shift CDF distribution by.
            minWeight : FLOAT
                A value between 0 and 1 that determines the minimum
                weight to assign to a diff value. 

            Returns
            --------
            node pair edge weight(s)

            """
            from scipy import stats
            # Transform from diff-space to gaussian-space
            if len(diff) == 1:
                QTGdiff = transformer.transform( np.reshape( np.array(diff), (1,1) ) );
            else:
                shape = np.shape(diff)
                QTGdiff = transformer.transform( np.reshape( np.array(diff), (np.size(diff), 1) ) )
                QTGdiff = np.reshape(QTGdiff, shape);
            # Get probablity in stretched distribution
            cdfCenter  = std*shiftFactor
            cdfStretch = std/shortenFactor
            CDF = stats.norm.cdf(QTGdiff, loc=cdfCenter, scale=cdfStretch)
            # Divide by probablity in normal distribution. This
            # scales probablility between 0-1.
            # Note that:
            #   S->1 for |value1 - value2|-> 0   and
            #   S->0 for |value1 - value2|-> inf
            Ss = ( (1-CDF) + minWeight )/(minWeight+1);

            return Ss
        
        Ss= complementCDF(diffAveGrd,
                          transformer = basins.Fields[fieldNum]['weightMethodPara']['qt'],
                          std = basins.Fields[fieldNum]['weightMethodPara']['qtDissSTD'],
                          shortenFactor = edgeWeightMethod['shortenFactor'],
                          shiftFactor = edgeWeightMethod['shiftFactor'],
                          minWeight = edgeWeightMethod['minWeight'])

        EC.plotHelper.plotGlobal(basins.lat, basins.lon, Ss,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_DQT_{}_weights.png".format(basins.Fields[fieldNum]['parameterUnit']),
                            cmapOpts={"cmap":cmc.batlow,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,
                                                    1]},
                            pltOpts={"valueType": "DQT",
                                     "valueUnits": "Weights",
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)



# F14: Community Detection of etopo (resolution tests) 1, 2, 4, 8 degrees.

#### Create different resolution sampled etopo grids 

In [ ]:
# Import necessary libraries
import ExoCcycle as EC
import matplotlib as mpl
import numpy as np
import os

spatialResolutions = [1, 2, 4, 8];

for resolution in spatialResolutions:
    # Create bathymetry object with body=... attributes
    # Try setting body='mars' | 'earth' | 'moon', 'Venus'
    body="Earth"
    planetBathy = EC.Bathymetry.BathyMeasured(body=body)

    # Create the directory if it doesn't exist
    directory_name = os.getcwd()+"/topographies/{}".format(planetBathy.model)
    os.makedirs(directory_name, exist_ok=True)

    # Download topography model (note that is must only be done once per body).
    # NOTE: UNCOMMENT THIS SECTION TO DOWNLOAD TOPOGRAPHY MODEL
    planetBathy.getTopo(os.getcwd(),
                      verbose=True);

    # Read topography. This method will generate a topography model netCDF4 file with 
    # new_resolution, in degrees. Note that the generated topography model will be 
    # cell registered (All calculations from here on out are in cell registered to
    # simplify codes and reduce data loss on conversions).
    planetBathy.readTopo(os.getcwd(),
                       new_resolution=resolution,
                       verbose=False);

    # Generate a bathymetry model base on a set of input methods and properties. The setSeaLelvel
    # method has multiple ways to fill topography with oceans. The two currently implemented at
    # the time of this JN creation are as follows:
    #     1) basinVolume : An option to define bathymetry by flooding topography with
    #                      basinVolume['uncompactedVol'] amount of ocean water, in m3.
    #     2) OceanArea : Option to define bathymetry by flooding topography until
    #                    oceanArea['area'], decimal percent, of global area is covered
    #                    with oceans.
    # 
    # Here, I use the OceanArea constraint and block the basinVolume constraint with comments.
    # 
    planetBathy.setSeaLevel(basinVolume = {"on":False, 'uncompactedVol':None},
                          oceanArea = {"on":True, "area":0.7},
                          isostaticCompensation = {"on":False}, verbose=False)

    # Note that in this basinVolume example the uncompactedVol was set to the oceanArea
    # bathymetry model's VOC, where oceanArea = {"on":True, "area":0.7}. 
    #
    # planetBathy.setSeaLevel(basinVolume = {"on":True, 'uncompactedVol':3.299187952154623e+17},
    #                       oceanArea = {"on":False, "area":0.7},
    #                       isostaticCompensation = {"on":False}, verbose=False)


    # Save bathymetry model as netCDF4. Note that models will be saved under the same root folder
    # that was supplied to the readTopo(...) method.
    # 
    # only the bathymetry array is saved in this file. Other relevant values are represented
    # as vectors (e.g., lat, lon, area-weights, global bathymetery distributions, etc) or
    # attributes (e.g., VOC, AOC, high latitude cutoff, etc).
    # 
    planetBathy.saveBathymetry()

    #
    # To see a full set of stored netCDF4 values, the user can uncomment the code below
    # 
    # planetBathy.readBathymetry()
    # print(planetBathy.bathync)
    # print(planetBathy.bathync.variables)

    # Plot bathymetry model
    plotRange = [np.nanmin(np.nanmin(planetBathy.bathymetry)), np.nanmean(planetBathy.bathymetry)+2*np.nanstd(planetBathy.bathymetry)]
    plotRange = [0, 6000];
    blues_cm = mpl.colormaps['Blues'].resampled(100)
    EC.plotHelper.plotGlobal(planetBathy.lat, planetBathy.lon, planetBathy.bathymetry,
                        outputDir = planetBathy.data_dir+"/bathymetries/{}".format(planetBathy.model),
                        fidName = "{}-setSeaLevel_Area0p7-resolution_{}deg.png".format(planetBathy.model, resolution),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "Bathymetry",
                                 "valueUnits": "m",
                                 "plotTitle":"{}".format(planetBathy.model),
                                 "plotZeroContour":True},
                        savePNG = True)


    # # Plot bathymetry model w/ bathymetry histograms
    # EC.plotHelper.plotGlobalwHist(planetBathy.lat, planetBathy.lon, planetBathy.bathymetry,
    #                         planetBathy.binEdges, planetBathy.bathymetryAreaDist_wHighlat, planetBathy.bathymetryAreaDist, planetBathy.highlatlat,
    #                         outputDir = planetBathy.data_dir+"/bathymetries/{}".format(planetBathy.model),
    #                         fidName = "{}-setSeaLevel_Area0p7.png".format(planetBathy.model),
    #                         cmapOpts={"cmap":blues_cm,
    #                                   "cbar-title":"cbar-title",
    #                                   "cbar-range":[np.nanmin(np.nanmin(planetBathy.bathymetry)),
    #                                                 np.nanmean(planetBathy.bathymetry)+2*np.nanstd(planetBathy.bathymetry)]},
    #                         pltOpts={"valueType": "Bathymetry",
    #                                  "valueUnits": "m",
    #                                  "plotTitle":"{}".format(planetBathy.model),
    #                                  "plotZeroContour":True},
    #                         savePNG = True)



#### Run community detections

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import copy as cp
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset
import cmcrameri.cm as cmc
import itertools

# Create nodeclustering object
from cdlib import evaluation
from cdlib import NodeClustering

#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden-Girvan-Newman"
communityDetectionMethod = "Leiden"

# Define basin merging criteria
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];

# Set of spatial resolutions to calculate basin for
spatialResolutions = [8, 4, 2, 1];
#spatialResolutions = [1];

# Define fields to plot weights for. Note that weights must be calculated
# for these fields.
fieldNums = ["Field1"]

# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt, spatialResolution in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, spatialResolutions):
    print(resolution, ensembleSize, minBasinCnt, spatialResolution)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt, spatialResolution in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, spatialResolutions):
    # Change data type to avoid errors
    spatialResolution = int(spatialResolution)
    minBasinCnt = int(minBasinCnt)
    ensembleSize = int(ensembleSize)
    
    # Set detection method
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results
    !mkdir -p figures/GMD_Manuscript/CodeOutputs/SpatialResolutionTests
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/SpatialResolutionTests/{}-PP'.format(communityDetectionMethod))
    print("Storing images in {}".format(fldName))
    
    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    


    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_{:0.1f}deg.nc".format(body, spatialResolution),
                             body=body);


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)

    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)
        
        
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    


    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    # Calculate area weighted average and standard deviation (for plotting)
#     areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
#                                                                                               LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
#                                                                                               LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)

    #########################
    ### Plot input fields ###
    #########################
    plotRange = [0, 6000];
    N = 1000
    blues_cm = mpl.colormaps['Blues'].resampled(N)
    #plotRange = [ave-1*std, ave+1*std];
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "boundaryColor":'k',
                                 "boundaryLinewidth":1.5,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    ###########################
    ### Plot DQT-CDF Values ###
    ###########################
    for fieldNum in fieldNums:
        #####################################################
        ####### Find average node neighbor difference #######
        #####################################################
        # Iterate over each node
        attrs = None;
        for node in basins.G.nodes:
            # Average node connection difference for each node
            temp = 0; cnt = 0;
            for conNode in basins.G.neighbors(node):
                temp+= np.abs( basins.G.nodes[conNode][fieldNum]-basins.G.nodes[node][fieldNum] )
                cnt+=1;
            try:
                diffAve = temp/cnt; 
            except:
                # If node has no connections. This rarely happens.
                #print(temp, cnt, basins.G.neighbors(node) )
                diffAve = 0
            # Collect average node neighbor difference property
            if attrs == None:
                attrs = {node: {"diffAve": diffAve}};
            else:
                attrs[node] = {"diffAve": diffAve};

        # Assign average node neighbor difference node property to graph
        G = nx.set_node_attributes(basins.G, attrs)

        # List values
        diffAve_values = list(basins.G.nodes(data="diffAve"))


        #####################################################
        ################ Interpolate to grid ################
        #####################################################
        def interp2regularGrid(basins, dataIrregular=None, mask=True):
            """
            interp2regularGrid method is used to interpolate data to
            a regular grid given an input of irregular spaced data.

            Parameters
            -----------
            dataIrregular : NUMPY ARRAY
                3XN numpy array with columns of longitude, latitude, magnitude.
                The default is None. This will make the function define the 
                dataIrregular variable with basinIDs.
            mask : STRING
                The path to a netCDF4 file that can be used to mask the result
                of interpolation. The default is None.

            Returns
            --------
            array : NUMPY ARRAY
                A 2nxn array that hold node properties for each corresponding
                entry in basins.lat and basins.lon. 

            """

            import copy as cp

            # Get basin IDs from network object.
            tmpValuesID  = nx.get_node_attributes(basins.G, "diffAve");
            tmpValuesPos = nx.get_node_attributes(basins.G, "pos");

            # Define an array to hold longitude, latitude, and basinID
            dataIrregular = np.zeros((len(tmpValuesPos), 3))

            # Iterate over all nodes so each node's longitude, latitude,
            # and basinID can be added to the dataIrregular array.
            for i in tmpValuesID:
                dataIrregular[i,:] = np.array([tmpValuesPos[i][1], tmpValuesPos[i][0], tmpValuesID[i]])

            # Define an array 2nxn to hold the basin IDs for the regular grid
            # on the surface of the a sphere (planet). 
            array = cp.deepcopy(basins.lat)

            # Define a mapping function that maps node indecies on a irregular grid
            # to those on the regular grid. This will speed up calculations if this
            # function is called more than once.

            # Iterate over all latitude and longitudes of the input grid.
            for i in range(len(basins.lat[:,0])):
                for j in range(len(basins.lat[0,:])):
                    # Find the distances from each regular grid point (i,j) to all
                    # irregular grid points.
                    x = EC.utils.haversine_distance(lat2= dataIrregular[:,1], lat1= basins.lat[i,j],
                                            lon2= dataIrregular[:,0], lon1= basins.lon[i,j],
                                            radius=1)

                    # Assign the nearest basin ID to element (i,j) 
                    array[i,j] = dataIrregular[np.argwhere(np.nanmin(x) == x)[0][0], 2]

            ## Apply the mask
            if mask:
                array[np.isnan(basins.maskValue)] = np.nan


            return array;

        # Interpolate from equal area grid to regular spaced latitude/longitude
        diffAveGrd = interp2regularGrid(basins, mask=True)

        #####################################################
        ####################### Plot ########################
        #####################################################
        # Transform using the QT
        DQT_zscore = cp.deepcopy(diffAveGrd);
        shape = np.shape(DQT_zscore)
        DQT_zscore = np.reshape(DQT_zscore, (np.size(DQT_zscore), 1) )
        DQT_zscoreNonNans = cp.deepcopy( DQT_zscore[~np.isnan(DQT_zscore)] )
        DQT_zscoreNonNans =  basins.Fields[fieldNum]['weightMethodPara']['qt'].transform( np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans), 1) ) )             
        DQT_zscore[~np.isnan(DQT_zscore)] = np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans),) )
        DQT_zscore = np.reshape(DQT_zscore, shape)


        # Plot using ExoCcycle plotGlobal function
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, DQT_zscore,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_DQT_{}.png".format(basins.Fields[fieldNum]['parameterUnit']),
                            cmapOpts={"cmap":cmc.batlow,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,
                                                    2]},
                            pltOpts={"valueType": "DQT",
                                     "valueUnits": "zscore",
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)

    # Plot using ExoCcycle plotGlobal function
    # Same as above except plots the weight values
    # excluding the distance dependence between nodes.
    # However, this should be negligible.
    def complementCDF(diff,
                      transformer,
                      std,
                      shortenFactor = edgeWeightMethod['shortenFactor'],
                      shiftFactor = edgeWeightMethod['shiftFactor'],
                      minWeight = edgeWeightMethod['minWeight']):
        """
        complementCDF is function used to calculate the weight of a node
        pair connection given the following inputs.

        Parameters
        -----------
        diff : FLOAT
            A node pair difference value from field of data.
        transformer : OBJECT
            Quantile transformation that is used to convert diff
            input into z-score value.
        std : FLAOT
            Standard devitaion of total field data input after
            being quantile transformed (i.e., this value should
            be ~0). 
        shortenFactor : FLOAT
            Factor to shorten CDF distribution by.
        shiftFactor : FLOAT
            Factor to shift CDF distribution by.
        minWeight : FLOAT
            A value between 0 and 1 that determines the minimum
            weight to assign to a diff value. 

        Returns
        --------
        node pair edge weight(s)

        """
        from scipy import stats
        # Transform from diff-space to gaussian-space
        if len(diff) == 1:
            QTGdiff = transformer.transform( np.reshape( np.array(diff), (1,1) ) );
        else:
            shape = np.shape(diff)
            QTGdiff = transformer.transform( np.reshape( np.array(diff), (np.size(diff), 1) ) )
            QTGdiff = np.reshape(QTGdiff, shape);
        # Get probablity in stretched distribution
        cdfCenter  = std*shiftFactor
        cdfStretch = std/shortenFactor
        CDF = stats.norm.cdf(QTGdiff, loc=cdfCenter, scale=cdfStretch)
        # Divide by probablity in normal distribution. This
        # scales probablility between 0-1.
        # Note that:
        #   S->1 for |value1 - value2|-> 0   and
        #   S->0 for |value1 - value2|-> inf
        Ss = ( (1-CDF) + minWeight )/(minWeight+1);

        return Ss

    Ss= complementCDF(diffAveGrd,
                      transformer = basins.Fields[fieldNum]['weightMethodPara']['qt'],
                      std = basins.Fields[fieldNum]['weightMethodPara']['qtDissSTD'],
                      shortenFactor = edgeWeightMethod['shortenFactor'],
                      shiftFactor = edgeWeightMethod['shiftFactor'],
                      minWeight = edgeWeightMethod['minWeight'])

    EC.plotHelper.plotGlobal(basins.lat, basins.lon, Ss,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_DQT_{}_weights.png".format(basins.Fields[fieldNum]['parameterUnit']),
                        cmapOpts={"cmap":cmc.batlow,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1]},
                        pltOpts={"valueType": "DQT",
                                 "valueUnits": "Weights",
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    

# F09, F17: Etopo versus reconstructed Bathymetry Community Detection

#### Leiden Detection + Post Processing with Etopo

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import copy as cp
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset
import cmcrameri.cm as cmc
import itertools

# Create nodeclustering object
from cdlib import evaluation
from cdlib import NodeClustering

#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden-Girvan-Newman"
communityDetectionMethod = "Leiden"

# Define basin merging criteria
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Define fields to plot weights for. Note that weights must be calculated
# for these fields.
fieldNums = ["Field2"]


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts):
    print(resolution, ensembleSize, minBasinCnt)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts):
    # Change data type to avoid errors
    minBasinCnt = int(minBasinCnt)
    ensembleSize = int(ensembleSize)
    
    # Set detection method
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results
    !mkdir -p figures/GMD_Manuscript/CodeOutputs/ReconVsMeasuredBathymetry
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/ReconVsMeasuredBathymetry/{}-PP'.format(communityDetectionMethod))
    print("Storing images in {}".format(fldName))
    
    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    


    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_1.0deg.nc".format(body),
                             body=body);


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)


    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)


    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)

    #########################
    ### Plot input fields ###
    #########################
    plotRange = [0, 6000];
    N = 1000
    blues_cm = mpl.colormaps['Blues'].resampled(N)
    #plotRange = [ave-1*std, ave+1*std];
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "boundaryColor":'k',
                                 "boundaryLinewidth":1.5,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    
    ###########################
    ### Plot DQT-CDF Values ###
    ###########################
    for fieldNum in fieldNums:
        #####################################################
        ####### Find average node neighbor difference #######
        #####################################################
        # Iterate over each node
        attrs = None;
        for node in basins.G.nodes:
            # Average node connection difference for each node
            temp = 0; cnt = 0;
            for conNode in basins.G.neighbors(node):
                temp+= np.abs( basins.G.nodes[conNode][fieldNum]-basins.G.nodes[node][fieldNum] )
                cnt+=1;
            try:
                diffAve = temp/cnt; 
            except:
                # If node has no connections. This rarely happens.
                #print(temp, cnt, basins.G.neighbors(node) )
                diffAve = 0
            # Collect average node neighbor difference property
            if attrs == None:
                attrs = {node: {"diffAve": diffAve}};
            else:
                attrs[node] = {"diffAve": diffAve};

        # Assign average node neighbor difference node property to graph
        G = nx.set_node_attributes(basins.G, attrs)

        # List values
        diffAve_values = list(basins.G.nodes(data="diffAve"))


        #####################################################
        ################ Interpolate to grid ################
        #####################################################
        def interp2regularGrid(basins, dataIrregular=None, mask=True):
            """
            interp2regularGrid method is used to interpolate data to
            a regular grid given an input of irregular spaced data.

            Parameters
            -----------
            dataIrregular : NUMPY ARRAY
                3XN numpy array with columns of longitude, latitude, magnitude.
                The default is None. This will make the function define the 
                dataIrregular variable with basinIDs.
            mask : STRING
                The path to a netCDF4 file that can be used to mask the result
                of interpolation. The default is None.

            Returns
            --------
            array : NUMPY ARRAY
                A 2nxn array that hold node properties for each corresponding
                entry in basins.lat and basins.lon. 

            """

            import copy as cp

            # Get basin IDs from network object.
            tmpValuesID  = nx.get_node_attributes(basins.G, "diffAve");
            tmpValuesPos = nx.get_node_attributes(basins.G, "pos");

            # Define an array to hold longitude, latitude, and basinID
            dataIrregular = np.zeros((len(tmpValuesPos), 3))

            # Iterate over all nodes so each node's longitude, latitude,
            # and basinID can be added to the dataIrregular array.
            for i in tmpValuesID:
                dataIrregular[i,:] = np.array([tmpValuesPos[i][1], tmpValuesPos[i][0], tmpValuesID[i]])

            # Define an array 2nxn to hold the basin IDs for the regular grid
            # on the surface of the a sphere (planet). 
            array = cp.deepcopy(basins.lat)

            # Define a mapping function that maps node indecies on a irregular grid
            # to those on the regular grid. This will speed up calculations if this
            # function is called more than once.

            # Iterate over all latitude and longitudes of the input grid.
            for i in range(len(basins.lat[:,0])):
                for j in range(len(basins.lat[0,:])):
                    # Find the distances from each regular grid point (i,j) to all
                    # irregular grid points.
                    x = EC.utils.haversine_distance(lat2= dataIrregular[:,1], lat1= basins.lat[i,j],
                                            lon2= dataIrregular[:,0], lon1= basins.lon[i,j],
                                            radius=1)

                    # Assign the nearest basin ID to element (i,j) 
                    array[i,j] = dataIrregular[np.argwhere(np.nanmin(x) == x)[0][0], 2]

            ## Apply the mask
            if mask:
                array[np.isnan(basins.maskValue)] = np.nan


            return array;

        # Interpolate from equal area grid to regular spaced latitude/longitude
        diffAveGrd = interp2regularGrid(basins, mask=True)

        #####################################################
        ####################### Plot ########################
        #####################################################
        # Transform using the QT
        DQT_zscore = cp.deepcopy(diffAveGrd);
        shape = np.shape(DQT_zscore)
        DQT_zscore = np.reshape(DQT_zscore, (np.size(DQT_zscore), 1) )
        DQT_zscoreNonNans = cp.deepcopy( DQT_zscore[~np.isnan(DQT_zscore)] )
        DQT_zscoreNonNans =  basins.Fields[fieldNum]['weightMethodPara']['qt'].transform( np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans), 1) ) )             
        DQT_zscore[~np.isnan(DQT_zscore)] = np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans),) )
        DQT_zscore = np.reshape(DQT_zscore, shape)


        # Plot using ExoCcycle plotGlobal function
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, DQT_zscore,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_DQT_{}.png".format(basins.Fields[fieldNum]['parameterUnit']),
                            cmapOpts={"cmap":cmc.batlow,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,
                                                    2]},
                            pltOpts={"valueType": "DQT",
                                     "valueUnits": "zscore",
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)

    # Plot using ExoCcycle plotGlobal function
    # Same as above except plots the weight values
    # excluding the distance dependence between nodes.
    # However, this should be negligible.
    def complementCDF(diff,
                      transformer,
                      std,
                      shortenFactor = edgeWeightMethod['shortenFactor'],
                      shiftFactor = edgeWeightMethod['shiftFactor'],
                      minWeight = edgeWeightMethod['minWeight']):
        """
        complementCDF is function used to calculate the weight of a node
        pair connection given the following inputs.

        Parameters
        -----------
        diff : FLOAT
            A node pair difference value from field of data.
        transformer : OBJECT
            Quantile transformation that is used to convert diff
            input into z-score value.
        std : FLAOT
            Standard devitaion of total field data input after
            being quantile transformed (i.e., this value should
            be ~0). 
        shortenFactor : FLOAT
            Factor to shorten CDF distribution by.
        shiftFactor : FLOAT
            Factor to shift CDF distribution by.
        minWeight : FLOAT
            A value between 0 and 1 that determines the minimum
            weight to assign to a diff value. 

        Returns
        --------
        node pair edge weight(s)

        """
        from scipy import stats
        # Transform from diff-space to gaussian-space
        if len(diff) == 1:
            QTGdiff = transformer.transform( np.reshape( np.array(diff), (1,1) ) );
        else:
            shape = np.shape(diff)
            QTGdiff = transformer.transform( np.reshape( np.array(diff), (np.size(diff), 1) ) )
            QTGdiff = np.reshape(QTGdiff, shape);
        # Get probablity in stretched distribution
        cdfCenter  = std*shiftFactor
        cdfStretch = std/shortenFactor
        CDF = stats.norm.cdf(QTGdiff, loc=cdfCenter, scale=cdfStretch)
        # Divide by probablity in normal distribution. This
        # scales probablility between 0-1.
        # Note that:
        #   S->1 for |value1 - value2|-> 0   and
        #   S->0 for |value1 - value2|-> inf
        Ss = ( (1-CDF) + minWeight )/(minWeight+1);

        return Ss

    Ss= complementCDF(diffAveGrd,
                      transformer = basins.Fields[fieldNum]['weightMethodPara']['qt'],
                      std = basins.Fields[fieldNum]['weightMethodPara']['qtDissSTD'],
                      shortenFactor = edgeWeightMethod['shortenFactor'],
                      shiftFactor = edgeWeightMethod['shiftFactor'],
                      minWeight = edgeWeightMethod['minWeight'])

    EC.plotHelper.plotGlobal(basins.lat, basins.lon, Ss,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_DQT_{}_weights.png".format(basins.Fields[fieldNum]['parameterUnit']),
                        cmapOpts={"cmap":cmc.batlow,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1]},
                        pltOpts={"valueType": "DQT",
                                 "valueUnits": "Weights",
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    
    # Create node cluster
    # Note that the small basin mergers are not inlcuded
    # in the LGNClusters. Only large basin mergers such that
    # small basin mergers results in X chosen basins.
    LeidenClusters=NodeClustering(communities=basins.Rcommunities,
                           graph=basins.G,
                           method_name="consensus_ledien",
                           method_parameters={
                               "resolution_parameter": resolution,
                               "runs": ensembleSize,
                               "distance_threshold": 0.3}
                          )

    LGNClusters=NodeClustering(communities=basins.communitiesFinal,
                           graph=basins.G,
                           method_name="consensus_ledien",
                           method_parameters={
                               "resolution_parameter": resolution,
                               "runs": ensembleSize,
                               "distance_threshold": 0.3}
                          )


    # Calculate community detection metrics
    for cluster, method in zip([LeidenClusters, LGNClusters], ["LeidenClusters", "LGNClusters"]):
        newman_girvan_modularity = evaluation.newman_girvan_modularity(basins.G, cluster)
        internal_edge_density = evaluation.internal_edge_density(basins.G, cluster)
        erdos_renyi_modularity= evaluation.erdos_renyi_modularity(basins.G, cluster)
        modularity_density    = evaluation.modularity_density(basins.G, cluster)
        avg_embeddedness      = evaluation.avg_embeddedness(basins.G, cluster)
        conductance           = evaluation.conductance(basins.G, cluster)
        surprise              = evaluation.surprise(basins.G, cluster)

        # Add community evaluation metrics to output
        readmetxt += "\n\nCommunity evaluation metrics ({}):\n".format(method);
        readmetxt += "newman_girvan_modularity:\t {}\n".format(newman_girvan_modularity.score)
        readmetxt += "erdos_renyi_modularity:\t\t {}\n".format(erdos_renyi_modularity.score)
        readmetxt += "modularity_density:\t\t {}\n".format(modularity_density.score)
        readmetxt += "internal_edge_density:\t\t {} +- {} (std)\n".format(internal_edge_density.score, internal_edge_density.std)
        readmetxt += "avg_embeddedness:\t\t {} +- {} (std)\n".format(avg_embeddedness.score, avg_embeddedness.std)
        readmetxt += "conductance:\t\t\t {} +- {} (std)\n".format(conductance.score, conductance.std)
        readmetxt += "surprise:\t\t\t {}\n".format(surprise.score)
        
    
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)


#### Leiden Detection + Post Processing with Reconstruction

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import copy as cp
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset
import cmcrameri.cm as cmc
import itertools

# Create nodeclustering object
from cdlib import evaluation
from cdlib import NodeClustering


####################################
### Reconstruction period inputs ###
####################################
ages = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80];
ages = [0, 60, 80]
agestr = [ str(age) for age in ages ];

# Unused unless communityDetectionMethod = "Leiden-Girvan-Newman"
minBasinCntV = [10]
minBasinCntVstr = [ str(cnt) for cnt in minBasinCntV ];



#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden-Girvan-Newman"
communityDetectionMethod = "Leiden"

# Define basin merging criteria
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Define fields to plot weights for. Note that weights must be calculated
# for these fields.
fieldNums = ["Field2"]


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt, age in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, ages):
    print(resolution, ensembleSize, minBasinCnt, age)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt, age in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, ages):
    # Change data type to avoid errors
    minBasinCnt = int(minBasinCnt);
    ensembleSize = int(ensembleSize);
    age = int(age);
    
    # Set detection method
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results
    !mkdir -p figures/GMD_Manuscript/CodeOutputs/ReconVsMeasuredBathymetry
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/ReconVsMeasuredBathymetry/Reconstruction_{}Ma-{}-PP'.format(age, communityDetectionMethod))
    print("Storing images in {}".format(fldName))
    
    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    


    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_1.0deg.nc".format(body),
                             body=body);
    ####################################
    ####### Add bathymetry field #######
    ####################################
    basins.addField(resolution = basins.Fields["Field1"]["resolution"],
                    dataGrid =  os.getcwd()+'/PNAS_Bogumil_Results/bathymetryNCFiles/Bathymetry_{}Ma.nc'.format(age),
                    parameter = "z",
                    parameterUnit = basins.Fields["Field1"]["parameterUnit"],
                    parameterName = basins.Fields["Field1"]["parameterName"])

    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field2"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)
    
    # Set field mask parameters
    fieldMaskParameter = {"usedField":0, "fliprl":False, "flipud":False}
    
    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        fieldMaskParameter = fieldMaskParameter,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)


    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)


    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field2"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field2"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    for fieldNum in fieldNums:
        # Read reconstructed bathymetry to plot it

        # Resample bathymetry at correct resolution and grid registration
        os.system("gmt grdsample {0} -Rd -rp -I{2}d, -G{1}".format("tempSimp_bathymetry.nc",
                                                                   "tempSimp_bathymetry_resampled.nc",
                                                                   basins.Fields[fieldNum]['resolution']))
        nc = Dataset("tempSimp_bathymetry_resampled.nc")
        bathymetry = nc["z"][:];
        lon = nc["lon"][:];
        lat = nc["lat"][:];
        nc.close()


        # Calculate area weighted average and standard deviation (for plotting)
        areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields[fieldNum]['resolution'],
                                                                                                  LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields[fieldNum]['resolution']],
                                                                                                  LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields[fieldNum]['resolution']])
        ave, std = EC.utils.weightedAvgAndStd(bathymetry, areaWeights)

        #########################
        ### Plot input fields ###
        #########################
        plotRange = [0, 6000];
        N = 1000
        blues_cm = mpl.colormaps['Blues'].resampled(N)
        #plotRange = [ave-1*std, ave+1*std];
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":blues_cm,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":plotRange},
                            pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)


        EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":blues_cm,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":plotRange},
                            pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "nanSolidPoly":True,
                                     "boundaryColor":'k',
                                     "boundaryLinewidth":1.5,
                                     "nanSolidPolyOutline":True,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)       
        
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    
    ###########################
    ### Plot DQT-CDF Values ###
    ###########################
    for fieldNum in fieldNums:
        #####################################################
        ####### Find average node neighbor difference #######
        #####################################################
        # Iterate over each node
        attrs = None;
        for node in basins.G.nodes:
            # Average node connection difference for each node
            temp = 0; cnt = 0;
            for conNode in basins.G.neighbors(node):
                temp+= np.abs( basins.G.nodes[conNode][fieldNum]-basins.G.nodes[node][fieldNum] )
                cnt+=1;
            try:
                diffAve = temp/cnt; 
            except:
                # If node has no connections. This rarely happens.
                #print(temp, cnt, basins.G.neighbors(node) )
                diffAve = 0
            # Collect average node neighbor difference property
            if attrs == None:
                attrs = {node: {"diffAve": diffAve}};
            else:
                attrs[node] = {"diffAve": diffAve};

        # Assign average node neighbor difference node property to graph
        G = nx.set_node_attributes(basins.G, attrs)

        # List values
        diffAve_values = list(basins.G.nodes(data="diffAve"))


        #####################################################
        ################ Interpolate to grid ################
        #####################################################
        def interp2regularGrid(basins, dataIrregular=None, mask=True):
            """
            interp2regularGrid method is used to interpolate data to
            a regular grid given an input of irregular spaced data.

            Parameters
            -----------
            dataIrregular : NUMPY ARRAY
                3XN numpy array with columns of longitude, latitude, magnitude.
                The default is None. This will make the function define the 
                dataIrregular variable with basinIDs.
            mask : STRING
                The path to a netCDF4 file that can be used to mask the result
                of interpolation. The default is None.

            Returns
            --------
            array : NUMPY ARRAY
                A 2nxn array that hold node properties for each corresponding
                entry in basins.lat and basins.lon. 

            """

            import copy as cp

            # Get basin IDs from network object.
            tmpValuesID  = nx.get_node_attributes(basins.G, "diffAve");
            tmpValuesPos = nx.get_node_attributes(basins.G, "pos");

            # Define an array to hold longitude, latitude, and basinID
            dataIrregular = np.zeros((len(tmpValuesPos), 3))

            # Iterate over all nodes so each node's longitude, latitude,
            # and basinID can be added to the dataIrregular array.
            for i in tmpValuesID:
                dataIrregular[i,:] = np.array([tmpValuesPos[i][1], tmpValuesPos[i][0], tmpValuesID[i]])

            # Define an array 2nxn to hold the basin IDs for the regular grid
            # on the surface of the a sphere (planet). 
            array = cp.deepcopy(basins.lat)

            # Define a mapping function that maps node indecies on a irregular grid
            # to those on the regular grid. This will speed up calculations if this
            # function is called more than once.

            # Iterate over all latitude and longitudes of the input grid.
            for i in range(len(basins.lat[:,0])):
                for j in range(len(basins.lat[0,:])):
                    # Find the distances from each regular grid point (i,j) to all
                    # irregular grid points.
                    x = EC.utils.haversine_distance(lat2= dataIrregular[:,1], lat1= basins.lat[i,j],
                                            lon2= dataIrregular[:,0], lon1= basins.lon[i,j],
                                            radius=1)

                    # Assign the nearest basin ID to element (i,j) 
                    array[i,j] = dataIrregular[np.argwhere(np.nanmin(x) == x)[0][0], 2]

            ## Apply the mask
            if mask:
                array[np.isnan(basins.maskValue)] = np.nan


            return array;

        # Interpolate from equal area grid to regular spaced latitude/longitude
        diffAveGrd = interp2regularGrid(basins, mask=True)

        #####################################################
        ####################### Plot ########################
        #####################################################
        # Transform using the QT
        DQT_zscore = cp.deepcopy(diffAveGrd);
        shape = np.shape(DQT_zscore)
        DQT_zscore = np.reshape(DQT_zscore, (np.size(DQT_zscore), 1) )
        DQT_zscoreNonNans = cp.deepcopy( DQT_zscore[~np.isnan(DQT_zscore)] )
        DQT_zscoreNonNans =  basins.Fields[fieldNum]['weightMethodPara']['qt'].transform( np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans), 1) ) )             
        DQT_zscore[~np.isnan(DQT_zscore)] = np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans),) )
        DQT_zscore = np.reshape(DQT_zscore, shape)


        # Plot using ExoCcycle plotGlobal function
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, DQT_zscore,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_DQT_{}.png".format(basins.Fields[fieldNum]['parameterUnit']),
                            cmapOpts={"cmap":cmc.batlow,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,
                                                    2]},
                            pltOpts={"valueType": "DQT",
                                     "valueUnits": "zscore",
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)

    # Plot using ExoCcycle plotGlobal function
    # Same as above except plots the weight values
    # excluding the distance dependence between nodes.
    # However, this should be negligible.
    def complementCDF(diff,
                      transformer,
                      std,
                      shortenFactor = edgeWeightMethod['shortenFactor'],
                      shiftFactor = edgeWeightMethod['shiftFactor'],
                      minWeight = edgeWeightMethod['minWeight']):
        """
        complementCDF is function used to calculate the weight of a node
        pair connection given the following inputs.

        Parameters
        -----------
        diff : FLOAT
            A node pair difference value from field of data.
        transformer : OBJECT
            Quantile transformation that is used to convert diff
            input into z-score value.
        std : FLAOT
            Standard devitaion of total field data input after
            being quantile transformed (i.e., this value should
            be ~0). 
        shortenFactor : FLOAT
            Factor to shorten CDF distribution by.
        shiftFactor : FLOAT
            Factor to shift CDF distribution by.
        minWeight : FLOAT
            A value between 0 and 1 that determines the minimum
            weight to assign to a diff value. 

        Returns
        --------
        node pair edge weight(s)

        """
        from scipy import stats
        # Transform from diff-space to gaussian-space
        if len(diff) == 1:
            QTGdiff = transformer.transform( np.reshape( np.array(diff), (1,1) ) );
        else:
            shape = np.shape(diff)
            QTGdiff = transformer.transform( np.reshape( np.array(diff), (np.size(diff), 1) ) )
            QTGdiff = np.reshape(QTGdiff, shape);
        # Get probablity in stretched distribution
        cdfCenter  = std*shiftFactor
        cdfStretch = std/shortenFactor
        CDF = stats.norm.cdf(QTGdiff, loc=cdfCenter, scale=cdfStretch)
        # Divide by probablity in normal distribution. This
        # scales probablility between 0-1.
        # Note that:
        #   S->1 for |value1 - value2|-> 0   and
        #   S->0 for |value1 - value2|-> inf
        Ss = ( (1-CDF) + minWeight )/(minWeight+1);

        return Ss

    Ss= complementCDF(diffAveGrd,
                      transformer = basins.Fields[fieldNum]['weightMethodPara']['qt'],
                      std = basins.Fields[fieldNum]['weightMethodPara']['qtDissSTD'],
                      shortenFactor = edgeWeightMethod['shortenFactor'],
                      shiftFactor = edgeWeightMethod['shiftFactor'],
                      minWeight = edgeWeightMethod['minWeight'])

    EC.plotHelper.plotGlobal(basins.lat, basins.lon, Ss,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_DQT_{}_weights.png".format(basins.Fields[fieldNum]['parameterUnit']),
                        cmapOpts={"cmap":cmc.batlow,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1]},
                        pltOpts={"valueType": "DQT",
                                 "valueUnits": "Weights",
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)


# F13: Etopo Basins (interior and boundary bathymetry distributions)

In [ ]:
####################################################
########### LGN Basin Detection on Etopo ###########
####################################################

#################################################
#################### Imports ####################
#################################################
import os
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset

# Create nodeclustering object
from cdlib import evaluation
from cdlib import NodeClustering

#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden-Girvan-Newman"

# Define basin merging criteria
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [0.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [5]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    print(resolution, ensembleSize, minBasinCnt)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage,
                       "njobs":4}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/CompositeMethod_Leiden-GirvanNewman-PP-Stats')
    print("Storing images in {}".format(fldName))
    
    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    


    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_1.0deg.nc".format(body),
                             body=body);


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)


    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)


    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)

    #########################
    ### Plot input fields ###
    #########################
    plotRange = [0, 6000];
    N = 1000
    blues_cm = mpl.colormaps['Blues'].resampled(N)
    #plotRange = [ave-1*std, ave+1*std];
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    
    EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":blues_cm,
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "boundaryColor":'k',
                                 "boundaryLinewidth":1.5,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    

    
############################################################################
########### Calculation/Plotting of Basin/Boundary Distributions ###########
############################################################################

# 12a. Calculate basin bathymetry parameters
# Note: the basin bathymetry distributions will be calculated
# with basins.bathymetry
basins.calculateBasinParameters(verbose=True, fldName=fldName, fieldNum="Field1")

# 12b. Calculate basin connectivity parameters
# Note: the connective distributions will be calculated
# with graph node bathymetry values.
basins.calculateBasinConnectivityParameters(verbose=True, fldName=fldName, fieldNum="Field1")

# 12c. Expand original bathymetry netCDF4 file by writting a new basin bathymetry
# netCDF4 file that also contains basin bathymetry parameters.
basins.saveCcycleParameter(verbose=True);






# F15, F16: Etopo Shelf vs Deep Marine Community Detection

#### Create etopo bathymetry netCDF4 at high resolution for shelf analysis 

In [ ]:
# Import necessary libraries
import ExoCcycle as EC
import matplotlib as mpl
import numpy as np
import os


# Set spatial resolutions
spatialResolutions = [0.3];

# Iterate over spatial resolutions to create each sampled etopo bathymetry model
for spatialResolution in spatialResolutions:
    # Create bathymetry object with body=... attributes
    # Try setting body='mars' | 'earth' | 'moon', 'Venus'
    body="Earth"
    planetBathy = EC.Bathymetry.BathyMeasured(body=body)

    # Create the directory if it doesn't exist
    directory_name = os.getcwd()+"/topographies/{}".format(planetBathy.model)
    os.makedirs(directory_name, exist_ok=True)

    # Download topography model (note that is must only be done once per body).
    # NOTE: UNCOMMENT THIS SECTION TO DOWNLOAD TOPOGRAPHY MODEL
    planetBathy.getTopo(os.getcwd(),
                      verbose=True);

    # Read topography. This method will generate a topography model netCDF4 file with 
    # new_resolution, in degrees. Note that the generated topography model will be 
    # cell registered (All calculations from here on out are in cell registered to
    # simplify codes and reduce data loss on conversions).
    planetBathy.readTopo(os.getcwd(),
                       new_resolution=spatialResolution,
                       verbose=False);

    # Generate a bathymetry model base on a set of input methods and properties. The setSeaLelvel
    # method has multiple ways to fill topography with oceans. The two currently implemented at
    # the time of this JN creation are as follows:
    #     1) basinVolume : An option to define bathymetry by flooding topography with
    #                      basinVolume['uncompactedVol'] amount of ocean water, in m3.
    #     2) OceanArea : Option to define bathymetry by flooding topography until
    #                    oceanArea['area'], decimal percent, of global area is covered
    #                    with oceans.
    # 
    # Here, I use the OceanArea constraint and block the basinVolume constraint with comments.
    # 
    planetBathy.setSeaLevel(basinVolume = {"on":False, 'uncompactedVol':None},
                          oceanArea = {"on":True, "area":0.7},
                          isostaticCompensation = {"on":False}, verbose=False)

    # Note that in this basinVolume example the uncompactedVol was set to the oceanArea
    # bathymetry model's VOC, where oceanArea = {"on":True, "area":0.7}. 
    #
    # planetBathy.setSeaLevel(basinVolume = {"on":True, 'uncompactedVol':3.299187952154623e+17},
    #                       oceanArea = {"on":False, "area":0.7},
    #                       isostaticCompensation = {"on":False}, verbose=False)


    # Save bathymetry model as netCDF4. Note that models will be saved under the same root folder
    # that was supplied to the readTopo(...) method.
    # 
    # only the bathymetry array is saved in this file. Other relevant values are represented
    # as vectors (e.g., lat, lon, area-weights, global bathymetery distributions, etc) or
    # attributes (e.g., VOC, AOC, high latitude cutoff, etc).
    # 
    planetBathy.saveBathymetry()



#### Prepare etopo data for community detection - i.e., split between pelagic and shelf regions

In [ ]:
from netCDF4 import Dataset
import os
import numpy as np
import ExoCcycle as EC
    
# Make directory for output files
!mkdir -p figures/GMD_Manuscript/CodeOutputs/shelf_noShelf_Communities/ncfiles
fldBase='figures/GMD_Manuscript/CodeOutputs/shelf_noShelf_Communities/ncfiles';

# Set spatial resolutions
spatialResolutions = [1, 0.3];

# Iterate over spatial resolutions to create each sampled etopo bathymetry model
for spatialResolution in spatialResolutions:
    # Input bathymetry
    if spatialResolution == int(spatialResolution):
        spatialResolution = int(spatialResolution)
    inputFile = os.getcwd()+"/bathymetries/Earth/Earth_resampled_{0:0.1f}deg.nc".format(spatialResolution)

    # Threshold depth [m]
    depthCut = 600

    # Make shelf bathymetry netCDF4
    #keepOnlySelectedVars(inputFile, fldBase+"/EtopoBathymetry.nc", keepVars=['lat', 'lon', 'bathymetry'])
    optionsShelf = {"inputFile":inputFile, "outputFile":fldBase+"/ShelfBathymetry_{0:0.1f}deg.nc".format(spatialResolution), "varName":"bathymetry", "threshold":depthCut, "gt":True}
    EC.utils.filterNc(optionsShelf)

    # # Make Deep ocean bathymetry netCDF4
    optionsDeep = {"inputFile":inputFile, "outputFile":fldBase+"/DeepBathymetry_{0:0.1f}deg.nc".format(spatialResolution), "varName":"bathymetry", "threshold":depthCut, "le":True}
    EC.utils.filterNc(optionsDeep)


### Run community detection as an alternative field (on Deep Bathymetry)

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import copy as cp
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset
import cmcrameri.cm as cmc

# Create nodeclustering object
from cdlib import evaluation
from cdlib import NodeClustering


###################################
### Different Bathymetry Models ###
###################################
fldBase='figures/GMD_Manuscript/CodeOutputs/shelf_noShelf_Communities/ncfiles';
bathymetryModelids = [0]
bathymetryModels = ["DeepBathymetry_{0:0.1f}deg.nc"]


# Set of spatial resolutions to calculate basin for
spatialResolutions = [1];

#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden-Girvan-Newman"
communityDetectionMethod = "Leiden"

# Define basin merging criteria
mergerPackageName = "Lite";

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [0.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Define fields to plot weights for. Note that weights must be calculated
# for these fields.
fieldNums = ["Field2"]


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt, bathymetryModelid in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, bathymetryModelids):
    print(resolution, ensembleSize, minBasinCnt, bathymetryModelid, bathymetryModels[int(bathymetryModelid)])


# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt, bathymetryModelid, spatialResolution in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, bathymetryModelids, spatialResolutions):
    # Change data type to avoid errors
    minBasinCnt = int(minBasinCnt);
    ensembleSize = int(ensembleSize);
    bathymetryModelid = int(bathymetryModelid);
    bathymetryModel = bathymetryModels[bathymetryModelid];
    if spatialResolution == int(spatialResolution):
        spatialResolution = int(spatialResolution)
    
    # Set detection method
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage,
                       "njobs":4}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results
    !mkdir -p figures/GMD_Manuscript/CodeOutputs/shelf_noShelf_Communities
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/shelf_noShelf_Communities/{0}-{1}-PP'.format(bathymetryModel.replace(".nc", "").format(spatialResolution), communityDetectionMethod))
    print("Storing images in {}".format(fldName))
    
    # Short readme text to write to folder with images
    readmetxt = "{} used to calculate basin boundaries.".format(bathymetryModel.replace(".nc", "").format(spatialResolution));
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    



    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{0}_resampled_{1:0.1f}deg.nc".format(body,spatialResolution),
                             body=body);
    ####################################
    # Add bathymetry field
    basins.addField(resolution = basins.Fields["Field1"]["resolution"],
                    dataGrid =  os.getcwd()+"/{0}/{1}".format(fldBase, bathymetryModel.format(spatialResolution)),
                    parameter = basins.Fields["Field1"]["parameter"],
                    parameterUnit = basins.Fields["Field1"]["parameterUnit"],
                    parameterName = basins.Fields["Field1"]["parameterName"])

    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field2"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)
    
    # Set field mask parameters
    fieldMaskParameter = {"usedField":0, "fliprl":False, "flipud":False}
    
    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        fieldMaskParameter = fieldMaskParameter,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)


    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    for fieldNum in fieldNums:
        # Read reconstructed bathymetry to plot it
        nc = Dataset(os.getcwd()+"/{0}/{1}".format(fldBase, bathymetryModel.format(spatialResolution)), 'r')
        
        lon = nc['lon'][:]
        lat = nc['lat'][:]
        bathymetry = nc["bathymetry"][:]
        nc.close()
        
        lon, lat = np.meshgrid(lon, lat);
        

        # Calculate area weighted average and standard deviation (for plotting)
        areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                                  LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                                  LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
        ave, std = EC.utils.weightedAvgAndStd(bathymetry, areaWeights)

        
        #########################
        ### Plot input fields ###
        #########################
        plotRange = [1000, 6000];
        N = 1000
        blues_cm = mpl.colormaps['Blues'].resampled(N)
        #plotRange = [ave-1*std, ave+1*std];
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":blues_cm,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":plotRange},
                            pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)


        EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":blues_cm,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":plotRange},
                            pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "nanSolidPoly":True,
                                     "boundaryColor":'k',
                                     "boundaryLinewidth":1.5,
                                     "nanSolidPolyOutline":True,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)
        
    #########################
    ### Plot Large Basins ###
    #########################
    # This function plots basins that are larger than half a percent of the total
    # shelf surface area.
    BasinIDAMod = EC.plotHelper.plotLargeBasins(basins,
                                           fieldNum="Field2",
                                           percentage=.5/100,
                                           fldName=os.getcwd()+"/"+fldName,
                                           savePNG=True,
                                           verbose=False)
    
        
    verbose = False
    if verbose:

        ###########################
        ### Plot DQT-CDF Values ###
        ###########################
        for fieldNum in fieldNums:
            #####################################################
            ####### Find average node neighbor difference #######
            #####################################################
            # Iterate over each node
            attrs = None;
            for node in basins.G.nodes:
                # Average node connection difference for each node
                temp = 0; cnt = 0;
                for conNode in basins.G.neighbors(node):
                    temp+= np.abs( basins.G.nodes[conNode][fieldNum]-basins.G.nodes[node][fieldNum] )
                    cnt+=1;
                try:
                    diffAve = temp/cnt; 
                except:
                    # If node has no connections. This rarely happens.
                    #print(temp, cnt, basins.G.neighbors(node) )
                    diffAve = 0
                # Collect average node neighbor difference property
                if attrs == None:
                    attrs = {node: {"diffAve": diffAve}};
                else:
                    attrs[node] = {"diffAve": diffAve};

            # Assign average node neighbor difference node property to graph
            G = nx.set_node_attributes(basins.G, attrs)

            # List values
            diffAve_values = list(basins.G.nodes(data="diffAve"))


            #####################################################
            ################ Interpolate to grid ################
            #####################################################
            def interp2regularGrid(basins, dataIrregular=None, mask=True):
                """
                interp2regularGrid method is used to interpolate data to
                a regular grid given an input of irregular spaced data.

                Parameters
                -----------
                dataIrregular : NUMPY ARRAY
                    3XN numpy array with columns of longitude, latitude, magnitude.
                    The default is None. This will make the function define the 
                    dataIrregular variable with basinIDs.
                mask : STRING
                    The path to a netCDF4 file that can be used to mask the result
                    of interpolation. The default is None.

                Returns
                --------
                array : NUMPY ARRAY
                    A 2nxn array that hold node properties for each corresponding
                    entry in basins.lat and basins.lon. 

                """

                import copy as cp

                # Get basin IDs from network object.
                tmpValuesID  = nx.get_node_attributes(basins.G, "diffAve");
                tmpValuesPos = nx.get_node_attributes(basins.G, "pos");

                # Define an array to hold longitude, latitude, and basinID
                dataIrregular = np.zeros((len(tmpValuesPos), 3))

                # Iterate over all nodes so each node's longitude, latitude,
                # and basinID can be added to the dataIrregular array.
                for i in tmpValuesID:
                    dataIrregular[i,:] = np.array([tmpValuesPos[i][1], tmpValuesPos[i][0], tmpValuesID[i]])

                # Define an array 2nxn to hold the basin IDs for the regular grid
                # on the surface of the a sphere (planet). 
                array = cp.deepcopy(basins.lat)

                # Define a mapping function that maps node indecies on a irregular grid
                # to those on the regular grid. This will speed up calculations if this
                # function is called more than once.

                # Iterate over all latitude and longitudes of the input grid.
                for i in range(len(basins.lat[:,0])):
                    for j in range(len(basins.lat[0,:])):
                        # Find the distances from each regular grid point (i,j) to all
                        # irregular grid points.
                        x = EC.utils.haversine_distance(lat2= dataIrregular[:,1], lat1= basins.lat[i,j],
                                                lon2= dataIrregular[:,0], lon1= basins.lon[i,j],
                                                radius=1)

                        # Assign the nearest basin ID to element (i,j) 
                        array[i,j] = dataIrregular[np.argwhere(np.nanmin(x) == x)[0][0], 2]

                ## Apply the mask
                if mask:
                    array[np.isnan(basins.maskValue)] = np.nan


                return array;

            # Interpolate from equal area grid to regular spaced latitude/longitude
            diffAveGrd = interp2regularGrid(basins, mask=True)

            #####################################################
            ####################### Plot ########################
            #####################################################
            # Transform using the QT
            DQT_zscore = cp.deepcopy(diffAveGrd);
            shape = np.shape(DQT_zscore)
            DQT_zscore = np.reshape(DQT_zscore, (np.size(DQT_zscore), 1) )
            DQT_zscoreNonNans = cp.deepcopy( DQT_zscore[~np.isnan(DQT_zscore)] )
            DQT_zscoreNonNans =  basins.Fields[fieldNum]['weightMethodPara']['qt'].transform( np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans), 1) ) )             
            DQT_zscore[~np.isnan(DQT_zscore)] = np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans),) )
            DQT_zscore = np.reshape(DQT_zscore, shape)


            # Plot using ExoCcycle plotGlobal function
            EC.plotHelper.plotGlobal(basins.lat, basins.lon, DQT_zscore,
                                outputDir = os.getcwd()+"/"+fldName,
                                fidName = "plotGlobal_DQT_{}.png".format(basins.Fields[fieldNum]['parameterUnit']),
                                cmapOpts={"cmap":cmc.batlow,
                                          "cbar-title":"cbar-title",
                                          "cbar-range":[0,
                                                        2]},
                                pltOpts={"valueType": "DQT",
                                         "valueUnits": "zscore",
                                         "plotTitle":"",
                                         "plotZeroContour":False,
                                         "transparent":True},
                                savePNG=True,
                                saveSVG=False)

        # Plot using ExoCcycle plotGlobal function
        # Same as above except plots the weight values
        # excluding the distance dependence between nodes.
        # However, this should be negligible.
        def complementCDF(diff,
                          transformer,
                          std,
                          shortenFactor = edgeWeightMethod['shortenFactor'],
                          shiftFactor = edgeWeightMethod['shiftFactor'],
                          minWeight = edgeWeightMethod['minWeight']):
            """
            complementCDF is function used to calculate the weight of a node
            pair connection given the following inputs.

            Parameters
            -----------
            diff : FLOAT
                A node pair difference value from field of data.
            transformer : OBJECT
                Quantile transformation that is used to convert diff
                input into z-score value.
            std : FLAOT
                Standard devitaion of total field data input after
                being quantile transformed (i.e., this value should
                be ~0). 
            shortenFactor : FLOAT
                Factor to shorten CDF distribution by.
            shiftFactor : FLOAT
                Factor to shift CDF distribution by.
            minWeight : FLOAT
                A value between 0 and 1 that determines the minimum
                weight to assign to a diff value. 

            Returns
            --------
            node pair edge weight(s)

            """
            from scipy import stats
            # Transform from diff-space to gaussian-space
            if len(diff) == 1:
                QTGdiff = transformer.transform( np.reshape( np.array(diff), (1,1) ) );
            else:
                shape = np.shape(diff)
                QTGdiff = transformer.transform( np.reshape( np.array(diff), (np.size(diff), 1) ) )
                QTGdiff = np.reshape(QTGdiff, shape);
            # Get probablity in stretched distribution
            cdfCenter  = std*shiftFactor
            cdfStretch = std/shortenFactor
            CDF = stats.norm.cdf(QTGdiff, loc=cdfCenter, scale=cdfStretch)
            # Divide by probablity in normal distribution. This
            # scales probablility between 0-1.
            # Note that:
            #   S->1 for |value1 - value2|-> 0   and
            #   S->0 for |value1 - value2|-> inf
            Ss = ( (1-CDF) + minWeight )/(minWeight+1);

            return Ss

        Ss= complementCDF(diffAveGrd,
                          transformer = basins.Fields[fieldNum]['weightMethodPara']['qt'],
                          std = basins.Fields[fieldNum]['weightMethodPara']['qtDissSTD'],
                          shortenFactor = edgeWeightMethod['shortenFactor'],
                          shiftFactor = edgeWeightMethod['shiftFactor'],
                          minWeight = edgeWeightMethod['minWeight'])

        EC.plotHelper.plotGlobal(basins.lat, basins.lon, Ss,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_DQT_{}_weights.png".format(basins.Fields[fieldNum]['parameterUnit']),
                            cmapOpts={"cmap":cmc.batlow,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,
                                                    1]},
                            pltOpts={"valueType": "DQT",
                                     "valueUnits": "Weights",
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)
    
    ######################################################
    ### Calculate metrics and write them to readme.txt ###
    ######################################################
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    readmetxt += ("\n\n" + metricText)
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)
        
        
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    






### Run community detection as an alternative field (on Shelf Bathymetry)

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import copy as cp
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset
import cmcrameri.cm as cmc
import itertools

# Create nodeclustering object
from cdlib import evaluation
from cdlib import NodeClustering


###################################
### Different Bathymetry Models ###
###################################
fldBase='figures/GMD_Manuscript/CodeOutputs/shelf_noShelf_Communities/ncfiles';
bathymetryModelids = [0]
bathymetryModels = ["ShelfBathymetry_{0:0.1f}deg.nc"]


# Set of spatial resolutions to calculate basin for
spatialResolutions = [0.3];

#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden-Girvan-Newman"
communityDetectionMethod = "Leiden"

# Define basin merging criteria
## This is the most similar merger to that used on global bathymetry (i.e. Lite)
mergerPackageName = "LiteShelf0,3degree";  # Given shelf area communities must imclude at least 408 nodes.

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;


# Resolution for quality function
resolutions = [0.15]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Define fields to plot weights for. Note that weights must be calculated
# for these fields.
fieldNums = ["Field2"]


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt, bathymetryModelid in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, bathymetryModelids):
    print(resolution, ensembleSize, minBasinCnt, bathymetryModelid, bathymetryModels[int(bathymetryModelid)])


# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt, bathymetryModelid, spatialResolution in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, bathymetryModelids, spatialResolutions):
    # Change data type to avoid errors
    minBasinCnt = int(minBasinCnt);
    ensembleSize = int(ensembleSize);
    bathymetryModelid = int(bathymetryModelid);
    bathymetryModel = bathymetryModels[bathymetryModelid];
    if spatialResolution == int(spatialResolution):
        spatialResolution = int(spatialResolution)
    
    # Set detection method
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage,
                       "njobs":4}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results
    !mkdir -p figures/GMD_Manuscript/CodeOutputs/shelf_noShelf_Communities
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/shelf_noShelf_Communities/{0}-{1}-PP'.format(bathymetryModel.replace(".nc", "").format(spatialResolution), communityDetectionMethod))
    print("Storing images in {}".format(fldName))
    
    # Short readme text to write to folder with images
    readmetxt = "{} used to calculate basin boundaries.".format(bathymetryModel.replace(".nc", "").format(spatialResolution));
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    



    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{0}_resampled_{1:0.1f}deg.nc".format(body,spatialResolution),
                             body=body);
    ####################################
    # Add bathymetry field
    basins.addField(resolution = basins.Fields["Field1"]["resolution"],
                    dataGrid =  os.getcwd()+"/{0}/{1}".format(fldBase, bathymetryModel.format(spatialResolution)),
                    parameter = basins.Fields["Field1"]["parameter"],
                    parameterUnit = basins.Fields["Field1"]["parameterUnit"],
                    parameterName = basins.Fields["Field1"]["parameterName"])

    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field2"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)
    
    # Set field mask parameters
    fieldMaskParameter = {"usedField":0, "fliprl":False, "flipud":False}
    
    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        fieldMaskParameter = fieldMaskParameter,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)


    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    for fieldNum in fieldNums:
        # Read reconstructed bathymetry to plot it
        nc = Dataset(os.getcwd()+"/{0}/{1}".format(fldBase, bathymetryModel.format(spatialResolution)), 'r')
        
        lon = nc['lon'][:]
        lat = nc['lat'][:]
        bathymetry = nc["bathymetry"][:]
        nc.close()
        
        lon, lat = np.meshgrid(lon, lat);
        

        # Calculate area weighted average and standard deviation (for plotting)
        areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                                  LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                                  LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
        ave, std = EC.utils.weightedAvgAndStd(bathymetry, areaWeights)

        
        #########################
        ### Plot input fields ###
        #########################
        plotRange = [0, 300];
        N = 1000
        blues_cm = mpl.colormaps['Blues'].resampled(N)
        #plotRange = [ave-1*std, ave+1*std];
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":blues_cm,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":plotRange},
                            pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)


        EC.plotHelper.plotGlobalwBoundaries(basins.lat, basins.lon, basins.bathymetry, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobalwboundaries_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":blues_cm,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":plotRange},
                            pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "nanSolidPoly":True,
                                     "boundaryColor":'k',
                                     "boundaryLinewidth":1.5,
                                     "nanSolidPolyOutline":True,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)
        
        
    #############################
    ##### Regional plotting #####
    #############################
    # SE Asia Shelves 1
    # SE Asia Shelves 2
    # SE Asia Shelves 3
    # ES South America Shelves 1
    regions = np.array([[100,150,-30,20],
                        [90,130,0,30],
                        [85,125,-10,25],
                        [-90, -35, -60, 20],
                       ])


    regionCnt=0
    for region in regions:
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotRegion{}.png".format(regionCnt),
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                            pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "-",
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "plotIntegerContours":True,
                                     "transparent":True,
                                     "coastlines":True,
                                     "region":region,
                                     "regionalZBoundaries":True},
                            savePNG=True,
                            saveSVG=False)

        if ave-1*std<0:
            lowerBound = 0;
        else:
            lowerBound = ave-1*std;
        #plotRange = [lowerBound, ave+1*std]
        plotRange = [0, 300]

        EC.plotHelper.plotGlobal(lat, lon, bathymetry,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotRegion{1}_{0}.png".format(basins.Fields[fieldNum]['parameterName'], regionCnt),
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":plotRange},
                            pltOpts={"valueType": "{0}".format(basins.Fields[fieldNum]['parameterName']),
                                     "valueUnits": "{}".format(basins.Fields[fieldNum]['parameterUnit']),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True,
                                     "coastlines":True,
                                     "region":region,
                                     "regionalZBoundaries":False},
                            savePNG=True,
                            saveSVG=False)
        regionCnt+=1
        
        
    verbose = False
    if verbose:

        ###########################
        ### Plot DQT-CDF Values ###
        ###########################
        for fieldNum in fieldNums:
            #####################################################
            ####### Find average node neighbor difference #######
            #####################################################
            # Iterate over each node
            attrs = None;
            for node in basins.G.nodes:
                # Average node connection difference for each node
                temp = 0; cnt = 0;
                for conNode in basins.G.neighbors(node):
                    temp+= np.abs( basins.G.nodes[conNode][fieldNum]-basins.G.nodes[node][fieldNum] )
                    cnt+=1;
                try:
                    diffAve = temp/cnt; 
                except:
                    # If node has no connections. This rarely happens.
                    #print(temp, cnt, basins.G.neighbors(node) )
                    diffAve = 0
                # Collect average node neighbor difference property
                if attrs == None:
                    attrs = {node: {"diffAve": diffAve}};
                else:
                    attrs[node] = {"diffAve": diffAve};

            # Assign average node neighbor difference node property to graph
            G = nx.set_node_attributes(basins.G, attrs)

            # List values
            diffAve_values = list(basins.G.nodes(data="diffAve"))


            #####################################################
            ################ Interpolate to grid ################
            #####################################################
            def interp2regularGrid(basins, dataIrregular=None, mask=True):
                """
                interp2regularGrid method is used to interpolate data to
                a regular grid given an input of irregular spaced data.

                Parameters
                -----------
                dataIrregular : NUMPY ARRAY
                    3XN numpy array with columns of longitude, latitude, magnitude.
                    The default is None. This will make the function define the 
                    dataIrregular variable with basinIDs.
                mask : STRING
                    The path to a netCDF4 file that can be used to mask the result
                    of interpolation. The default is None.

                Returns
                --------
                array : NUMPY ARRAY
                    A 2nxn array that hold node properties for each corresponding
                    entry in basins.lat and basins.lon. 

                """

                import copy as cp

                # Get basin IDs from network object.
                tmpValuesID  = nx.get_node_attributes(basins.G, "diffAve");
                tmpValuesPos = nx.get_node_attributes(basins.G, "pos");

                # Define an array to hold longitude, latitude, and basinID
                dataIrregular = np.zeros((len(tmpValuesPos), 3))

                # Iterate over all nodes so each node's longitude, latitude,
                # and basinID can be added to the dataIrregular array.
                for i in tmpValuesID:
                    dataIrregular[i,:] = np.array([tmpValuesPos[i][1], tmpValuesPos[i][0], tmpValuesID[i]])

                # Define an array 2nxn to hold the basin IDs for the regular grid
                # on the surface of the a sphere (planet). 
                array = cp.deepcopy(basins.lat)

                # Define a mapping function that maps node indecies on a irregular grid
                # to those on the regular grid. This will speed up calculations if this
                # function is called more than once.

                # Iterate over all latitude and longitudes of the input grid.
                for i in range(len(basins.lat[:,0])):
                    for j in range(len(basins.lat[0,:])):
                        # Find the distances from each regular grid point (i,j) to all
                        # irregular grid points.
                        x = EC.utils.haversine_distance(lat2= dataIrregular[:,1], lat1= basins.lat[i,j],
                                                lon2= dataIrregular[:,0], lon1= basins.lon[i,j],
                                                radius=1)

                        # Assign the nearest basin ID to element (i,j) 
                        array[i,j] = dataIrregular[np.argwhere(np.nanmin(x) == x)[0][0], 2]

                ## Apply the mask
                if mask:
                    array[np.isnan(basins.maskValue)] = np.nan


                return array;

            # Interpolate from equal area grid to regular spaced latitude/longitude
            diffAveGrd = interp2regularGrid(basins, mask=True)
            #diffAveGrd = EC.utils.interp2regularGrid(basins, mask=True, propertyName='diffAve')

            #####################################################
            ####################### Plot ########################
            #####################################################
            # Transform using the QT
            DQT_zscore = cp.deepcopy(diffAveGrd);
            shape = np.shape(DQT_zscore)
            DQT_zscore = np.reshape(DQT_zscore, (np.size(DQT_zscore), 1) )
            DQT_zscoreNonNans = cp.deepcopy( DQT_zscore[~np.isnan(DQT_zscore)] )
            DQT_zscoreNonNans =  basins.Fields[fieldNum]['weightMethodPara']['qt'].transform( np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans), 1) ) )             
            DQT_zscore[~np.isnan(DQT_zscore)] = np.reshape(DQT_zscoreNonNans, (len(DQT_zscoreNonNans),) )
            DQT_zscore = np.reshape(DQT_zscore, shape)


            # Plot using ExoCcycle plotGlobal function
            EC.plotHelper.plotGlobal(basins.lat, basins.lon, DQT_zscore,
                                outputDir = os.getcwd()+"/"+fldName,
                                fidName = "plotGlobal_DQT_{}.png".format(basins.Fields[fieldNum]['parameterUnit']),
                                cmapOpts={"cmap":cmc.batlow,
                                          "cbar-title":"cbar-title",
                                          "cbar-range":[0,
                                                        2]},
                                pltOpts={"valueType": "DQT",
                                         "valueUnits": "zscore",
                                         "plotTitle":"",
                                         "plotZeroContour":False,
                                         "transparent":True},
                                savePNG=True,
                                saveSVG=False)

        # Plot using ExoCcycle plotGlobal function
        # Same as above except plots the weight values
        # excluding the distance dependence between nodes.
        # However, this should be negligible.
        def complementCDF(diff,
                          transformer,
                          std,
                          shortenFactor = edgeWeightMethod['shortenFactor'],
                          shiftFactor = edgeWeightMethod['shiftFactor'],
                          minWeight = edgeWeightMethod['minWeight']):
            """
            complementCDF is function used to calculate the weight of a node
            pair connection given the following inputs.

            Parameters
            -----------
            diff : FLOAT
                A node pair difference value from field of data.
            transformer : OBJECT
                Quantile transformation that is used to convert diff
                input into z-score value.
            std : FLAOT
                Standard devitaion of total field data input after
                being quantile transformed (i.e., this value should
                be ~0). 
            shortenFactor : FLOAT
                Factor to shorten CDF distribution by.
            shiftFactor : FLOAT
                Factor to shift CDF distribution by.
            minWeight : FLOAT
                A value between 0 and 1 that determines the minimum
                weight to assign to a diff value. 

            Returns
            --------
            node pair edge weight(s)

            """
            from scipy import stats
            # Transform from diff-space to gaussian-space
            if len(diff) == 1:
                QTGdiff = transformer.transform( np.reshape( np.array(diff), (1,1) ) );
            else:
                shape = np.shape(diff)
                QTGdiff = transformer.transform( np.reshape( np.array(diff), (np.size(diff), 1) ) )
                QTGdiff = np.reshape(QTGdiff, shape);
            # Get probablity in stretched distribution
            cdfCenter  = std*shiftFactor
            cdfStretch = std/shortenFactor
            CDF = stats.norm.cdf(QTGdiff, loc=cdfCenter, scale=cdfStretch)
            # Divide by probablity in normal distribution. This
            # scales probablility between 0-1.
            # Note that:
            #   S->1 for |value1 - value2|-> 0   and
            #   S->0 for |value1 - value2|-> inf
            Ss = ( (1-CDF) + minWeight )/(minWeight+1);

            return Ss

        Ss= complementCDF(diffAveGrd,
                          transformer = basins.Fields[fieldNum]['weightMethodPara']['qt'],
                          std = basins.Fields[fieldNum]['weightMethodPara']['qtDissSTD'],
                          shortenFactor = edgeWeightMethod['shortenFactor'],
                          shiftFactor = edgeWeightMethod['shiftFactor'],
                          minWeight = edgeWeightMethod['minWeight'])

        EC.plotHelper.plotGlobal(basins.lat, basins.lon, Ss,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_DQT_{}_weights.png".format(basins.Fields[fieldNum]['parameterUnit']),
                            cmapOpts={"cmap":cmc.batlow,
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,
                                                    1]},
                            pltOpts={"valueType": "DQT",
                                     "valueUnits": "Weights",
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)
    
    ######################################################
    ### Calculate metrics and write them to readme.txt ###
    ######################################################
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    
    readmetxt += ("\n\n" + metricText)

    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)
        
        
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    



smallBasinThresholdkm2 = basins.AOC*(np.max(mergerPackage['mergeSmallBasins']['threshold'])/100)
smallBasinMinNodes     = np.floor( smallBasinThresholdkm2/basins.G.nodes[0]['areaWeightm2'] )

print(f"The small basin merger package requires no basin be smaller than {smallBasinThresholdkm2:0.2f} km^2.")
print(f"The small basin merger package requires no basin have no less than {smallBasinMinNodes:0.0f} nodes.")



### Test plotting other regions by changing regions array variable

In [ ]:
#############################
##### Regional plotting #####
#############################
# SE Asia Shelves 1
# SE Asia Shelves 2
# ES South America Shelves 1
regions = np.array([[85,125,-10,25]])

regionCnt-=1
for region in regions:
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotRegion{}.png".format(regionCnt),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True,
                                 "coastlines":True,
                                 "region":region,
                                 "regionalZBoundaries":True},
                        savePNG=True,
                        saveSVG=False)

    if ave-1*std<0:
        lowerBound = 0;
    else:
        lowerBound = ave-1*std;
        
    #plotRange = [lowerBound, ave+1*std]
    plotRange = [0,300]

    EC.plotHelper.plotGlobal(lat, lon, bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotRegion{1}_{0}.png".format(basins.Fields[fieldNum]['parameterName'], regionCnt),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":plotRange},
                        pltOpts={"valueType": "{0}".format(basins.Fields[fieldNum]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields[fieldNum]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True,
                                 "coastlines":True,
                                 "region":region,
                                 "regionalZBoundaries":False},
                        savePNG=True,
                        saveSVG=False)
    regionCnt+=1




### Determine equivalent small basin threshold ($\chi$) values for systems with different node counts (e.g., shelf only analysis)

In [ ]:
# Lite node counts for global ocean system
AOCOcean = 361e12
resolution_at_1deg = basins.G.nodes[0]['areaWeightm2']/.3
percentages = [0.1, 0.5] 
SmallestBasin1 = AOCOcean*(percentages[0]/100)
SmallestBasin2 = AOCOcean*(percentages[1]/100)
print(f"(Whole Ocean) The small basin merger package requires no basin have no less than {SmallestBasin1/resolution_at_1deg:0.0f} nodes.")
print(f"(Whole Ocean) The small basin merger package requires no basin have no less than {SmallestBasin2/resolution_at_1deg:0.0f} nodes.")


# Lite node counts for shelf ocean system
percentagesShelf = [0.03, 0.15]
smallBasinThresholdkm2 = basins.AOC*(percentagesShelf[0])/100
smallBasinMinNodes     = np.floor( smallBasinThresholdkm2/basins.G.nodes[0]['areaWeightm2'] )
print(f"\n(Shelf) The small basin merger package requires no basin be smaller than {smallBasinThresholdkm2:0.2f} km^2.")
print(f"(Shelf) The small basin merger package requires no basin have no less than {smallBasinMinNodes:0.0f} nodes.")

smallBasinThresholdkm2 = basins.AOC*(percentagesShelf[1])/100
smallBasinMinNodes     = np.floor( smallBasinThresholdkm2/basins.G.nodes[0]['areaWeightm2'] )

print(f"(Shelf) The small basin merger package requires no basin be smaller than {smallBasinThresholdkm2:0.2f} km^2.")
print(f"(Shelf) The small basin merger package requires no basin have no less than {smallBasinMinNodes:0.0f} nodes.")


print("\u03C7 = {0} for global ocean are equivalent to \u03C7 = {1} for global shelf analysis.".format(percentages, percentagesShelf))



# Walltime Run \& Basin Count 1: Louvain + No small basin merger

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset
import time


#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method

#communityDetectionMethod = "Louvain"
#communityDetectionMethod = "Leiden"
communityDetectionMethod = "Girvan-Newman"
#communityDetectionMethod = "Leiden-Girvan-Newman"

# Define basin merging criteria
mergerPackageName = 'Lite'; # ['threshold'] = [0]
#mergerPackageName = 'Lite';

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [5]

# Define spatial resolutions
spatialResolutions = [10, 8, 4];
# spatialResolutions = [10,8,4,1];

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [1];

# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.

for resolution, ensembleSize, minBasinCnt, spatialResolution in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, spatialResolutions):
    print(resolution, ensembleSize, minBasinCnt, spatialResolution)

# Run for multiple resolution, ensembleSize, and minBasinCnt
# for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
for resolution, ensembleSize, minBasinCnt, spatialResolution in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, spatialResolutions):
    # Change data type to avoid errors
    spatialResolution = int(spatialResolution)
    minBasinCnt = int(minBasinCnt)
    ensembleSize = int(ensembleSize)
    
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage,
                       "njobs":1}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results    
    !mkdir -p figures/GMD_Manuscript/CodeOutputs/SpatialResolutionTestsWallTimes
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/SpatialResolutionTestsWallTimes/{0}_{1}'.format(communityDetectionMethod,spatialResolution))
    print("Storing images in {}".format(fldName))

    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    if not (communityDetectionMethod == "Girvan-Newman"):
        readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)
    
    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]
    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_{:0.1f}deg.nc".format(body, spatialResolution),
                             body=body);


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    startTime = time.time()
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)
    endTime = time.time()
    print("Detection Time: {} seconds".format(endTime-startTime))


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)
    
    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    readmetxt += "\nDetection Time: {} seconds\n".format(endTime-startTime)
    
    # Get community detection metrics
    if not (communityDetectionMethod == "Girvan-Newman"):
        metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                             resolution=resolution,
                                                             ensembleSize=ensembleSize,
                                                             distance_threshold=0.3)
        readmetxt += ("\n\n" + metricText)
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)



    #####################################
    ### Plot results of community IDs ###
    #####################################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    


    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)  



    #########################
    ### Plot input fields ###
    #########################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[ave-1*std,
                                                ave+1*std]},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_2_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[-1,
                                                1],
                                  "cbar-levels":np.array([-1,0, .5, 1]),
                                  "cbar-intervals":np.array([-1, 0, .5, 1]),
                                  "cbar-level-names":["No", "Poor", "Good"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)

# Theoretical community detection runtime scaling

Given runs with of Louvain, Leiden, Girvan-Newman, and Leiden-Girvan-Newman with different spatial resolution (10, 8, 4, 1 degree) the below node counts and runtimes can be collected to do the below analysis.

In [ ]:
import numpy as np

# Assign graph characteristics
spatialResolutions = [10,8,4,1];
nodeCnt = np.array([284, 527, 1603, 24441])
edgeCnt = 2*nodeCnt


# Define node/edge scaling with increasing resolution steps
nodeScaling = nodeCnt/nodeCnt[0]
edgeScaling = edgeCnt/edgeCnt[0]


# Define calculation walltimes (calculated with simulations in pervious cell)
LouvainTimes = np.array([0.6, 1, 2.6, 55.6])
LeidenTimes = np.array([0.7, 0.8, 2.6, 55.6])
GNTimes = np.array([8.5, 40.2, 0, 0])

# Define functions for theoretical scaling for increasing node and edge count
def fncLouvainPredictedTimes(n,m):
    return n*np.log(n)
def fncGNPredictedTimes(n,m):
    return n*(m**2)

# Calculate the predicted walltimes based on coarsest model walltime + theoretical scaling
LouvainPredictedTimes = LouvainTimes[0]*fncLouvainPredictedTimes(nodeCnt,edgeCnt)/fncLouvainPredictedTimes(nodeCnt,edgeCnt)[0]
LeidenPredictedTimes = LeidenTimes[0]*fncLouvainPredictedTimes(nodeCnt,edgeCnt)/fncLouvainPredictedTimes(nodeCnt,edgeCnt)[0]
GNPredictedTimes = GNTimes[0]*fncGNPredictedTimes(nodeCnt,edgeCnt)/fncGNPredictedTimes(nodeCnt,edgeCnt)[0]


# Report the predicted walltimes based on coarsest model walltime + theoretical scaling
print("Runtimes for Louvain Model predicted with:")
print("10 degree {0:2.1f}s; 8 degree {1:2.1f}s; 4 degree {2:2.1f}s; 1 degree {3:2.1f}s\n".format(
    LouvainPredictedTimes[0],
    LouvainPredictedTimes[1],
    LouvainPredictedTimes[2],
    LouvainPredictedTimes[3]) )

print("Runtimes for Leiden Model predicted with:")
print("10 degree {0:2.1f}s; 8 degree {1:2.1f}s; 4 degree {2:2.1f}s; 1 degree {3:2.1f}s\n".format(
    LeidenPredictedTimes[0],
    LeidenPredictedTimes[1],
    LeidenPredictedTimes[2],
    LeidenPredictedTimes[3]) )

print("Runtimes for Girvan-Newman Model predicted with:")
print("10 degree {0:2.1f}s; 8 degree {1:2.1f}s; 4 degree {2:2.1f} min; 1 degree {3:2.1f} days\n".format(
    GNPredictedTimes[0],
    GNPredictedTimes[1],
    GNPredictedTimes[2]/60,
    GNPredictedTimes[3]/60/60/24) )


# F08: Global silhouette score variation w.r.t. ensemble size

### Calculate models

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset
import time


#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method

#communityDetectionMethod = "Louvain"
communityDetectionMethod = "Leiden"
#communityDetectionMethod = "Girvan-Newman"
#communityDetectionMethod = "Leiden-Girvan-Newman"

# Define basin merging criteria
mergerPackageName = 'Lite'; # ['threshold'] = [0]
#mergerPackageName = 'Lite';

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [5]

# Define spatial resolutions
spatialResolutions = [1];

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
setEnsembleSizes = [1,3,5,10,20,30,40,50];
nsimulation = 20
ensembleSizes = []
for ensembleSize in setEnsembleSizes:
    for j in range(nsimulation):
        ensembleSizes.append(ensembleSize)

# Set initiation to false, this tells defineBasins not to recalculate
# 1) graph (nodes and connections)
# 2) node values from the field
# 3) node edge weights node values and used method
# Only set to false if this is not the first time the cell is being run in this kernel.
initiation = True;

# Option to output plots or only readme.txt used for analysis
makePlots = False;

# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.

for resolution, ensembleSize, minBasinCnt, spatialResolution in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, spatialResolutions):
    print(resolution, ensembleSize, minBasinCnt, spatialResolution)

# Run for multiple resolution, ensembleSize, and minBasinCnt
# for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
for resolution, ensembleSize, minBasinCnt, spatialResolution in EC.utils.combine_lists(resolutions, ensembleSizes, minBasinCnts, spatialResolutions):
    # Change data type to avoid errors
    spatialResolution = int(spatialResolution)
    minBasinCnt = int(minBasinCnt)
    ensembleSize = int(ensembleSize)
    
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "constantSeeds":False,
                       "minBasinLargerThanSmallMergers":True,
                       "mergerPackage":mergerPackage,
                       "njobs":8}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results    
    !mkdir -p figures/GMD_Manuscript/CodeOutputs/EnsembleSilhouetteScores
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/EnsembleSilhouetteScores/EnsembleSize{0}_Resolution-{1}deg'.format(ensembleSize,spatialResolution))
    print("Storing images in {}".format(fldName))

    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    

    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################
    # Create basin object
    if initiation:
        body = ["Earth", "Mars", "Venus", "Moon"]
        body = body[0]
        basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                                 filename="{}_resampled_{:0.1f}deg.nc".format(body, spatialResolution),
                                 body=body);


        # Assign fields to use in community detection
        basins.useFields(fieldList=np.array(["Field1"]))

        # Show all fields stored in basins object
        basins.getFields(usedFields = False)

        # Show all fields stored in basins object that will be used
        # for community detection.
        basins.getFields(usedFields = True)
        

    #########################################
    ### Run Community Detection Algorithm ###
    #########################################

    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    startTime = time.time()
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False,
                        initiation=initiation)
    endTime = time.time()
    print("Detection Time: {} seconds".format(endTime-startTime))
    
    # Set initiation to false, this tells defineBasins not to recalculate
    # 1) graph (nodes and connections)
    # 2) node values from the field
    # 3) node edge weights node values and used method
    initiation = False

    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)
    
    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    readmetxt += "\nDetection Time: {} seconds\n".format(endTime-startTime)
    
    # Get community detection metrics
    if not (communityDetectionMethod == "Girvan-Newman"):
        metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                             resolution=resolution,
                                                             ensembleSize=ensembleSize,
                                                             distance_threshold=0.3)
        readmetxt += ("\n\n" + metricText)
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)



    #####################################
    ### Plot results of community IDs ###
    #####################################
    if makePlots:
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal.png",
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                            pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "-",
                                     "plotTitle":"",
                                     "mesh":True,
                                     "plotZeroContour":False,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)



        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_Contour.png",
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                            pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "-",
                                     "plotTitle":"",
                                     "mesh":False,
                                     "coastlines":False,
                                     "nanSolidPoly":True,
                                     "nanSolidPolyOutline":True,
                                     "plotZeroContour":False,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)  



        #########################
        ### Plot input fields ###
        #########################
        EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[ave-1*std,
                                                    ave+1*std]},
                            pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                     "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)


        ##############################
        ### Plot Silhouette fields ###
        ##############################
        EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[0,
                                                    1],
                                      "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                      "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                      "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                            pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                     "valueUnits": "{}".format("-"),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "nanSolidPoly":True,
                                     "nanSolidPolyOutline":True,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)

        EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                            outputDir = os.getcwd()+"/"+fldName,
                            fidName = "plotGlobal_silhouette_2_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                            cmapOpts={"cmap":"jet",
                                      "cbar-title":"cbar-title",
                                      "cbar-range":[-1,
                                                    1],
                                      "cbar-levels":np.array([-1,0, .5, 1]),
                                      "cbar-intervals":np.array([-1, 0, .5, 1]),
                                      "cbar-level-names":["No", "Poor", "Good"]},
                            pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                     "valueUnits": "{}".format("-"),
                                     "plotTitle":"",
                                     "plotZeroContour":False,
                                     "nanSolidPoly":True,
                                     "nanSolidPolyOutline":True,
                                     "plotIntegerContours":True,
                                     "transparent":True},
                            savePNG=True,
                            saveSVG=False)

### Plot model results

In [ ]:
# Import used libraries
from __future__ import annotations

import re
import math
from pathlib import Path
from typing import Iterable, Dict, List, Tuple, Optional
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

_SILHOUETTE_RE = re.compile(
    r"""^Silhouette\ value:\s*
        (?P<val>[+-]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)
        \s*(?:±|\+−|\+-|\u00B1|\s*\\pm\s*)
        \s*(?P<std>[+-]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?)
        \s*$
    """,
    re.VERBOSE,
)


def _parse_silhouette(readme_path: Path) -> Optional[Tuple[float, float]]:
    """
    _parse_silhouette reads value and std pair from the readme.txt outputed
    by basin detections above.
    
    Returns  
    --------
    value : FLOAT
        silhouette value from readme.txt, or None if not found.
    reported_std : FLOAT
        silhouette std value from readme.txt, or None if not found.
    """
    try:
        for line in readme_path.read_text(encoding="utf-8", errors="ignore").splitlines():
            line = line.strip()
            m = _SILHOUETTE_RE.match(line)
            if m:
                return float(m.group("val")), float(m.group("std"))
    except Exception:
        pass
    return None


def weightedAverageUncertainty(values, std):
    '''
    weightedAverageUncertainty is used to do error propagation
    given input values and their uncertainty.
    
    
    Parameters
    -----------
    values : NDARRAY
        Measurement value.
    
    std : NDARRAY
        Measurement uncertainty value.
        
    Return
    -------
    xw : FLOAT
        Uncertainty weighted average.
    
    stdw : FLOAT
        Uncertainty weighted uncertainty for xw.
    '''
    import numpy as np
    
    # Define weight based on measurement uncertainty
    w    = 1/(std**2);
    
    # Define the measurement uncertainty weighted uncertainty
    stdw = 1/np.sqrt(np.sum(w))

    # Define the measurement uncertainty weighted average
    xw   = np.sum(w*values)/np.sum(w)
    
    # Return values
    return xw, stdw


def summarize_silhouette_by_ensemble(
    base_path: Path | str,
    ensemble_sizes: Iterable[int],
    *,
    file_name: str = "readme.txt",
    pattern_prefix: str = "EnsembleSize",      # e.g., EnsembleSize{ES}_Resolution-1deg_{RUN}
    pattern_middle: str = "_Resolution-1deg_", # the fixed chunk between ES and run index
) -> pd.DataFrame:
    """
    summarize_silhouette_by_ensemble scans directories like:
        base_path / f"EnsembleSize{ES}_Resolution-1deg_*/readme.txt"
    extract 'Silhouette value: X ± Y', and compute per-ensemble-size aggregates.

    Parameters
    ----------
    base_path : PATH | STRING
        Root directory containing EnsembleSize* folders.
    ensemble_sizes : ITERABLE[INT]
        Which ensemble sizes to aggregate.
    file_name : STRING
        Name of the file to parse in each run directory (default 'readme.txt').
    pattern_prefix : STRING
        Prefix before the ensemble size in the directory name.
    pattern_middle : STRING
        Fixed text between ensemble size and the run index.

    Returns
    -------
    pd.DataFrame
        One row per ensemble size with:
            - ensemble_size
            - n_runs_parsed
            - value_mean             (mean of the absolute silhouette values across runs)
            - value_std_across_runs  (standard deviation of values across runs)
            - reported_std_mean      (mean of the reported std after '±' across runs)
            - reported_std_std       (std of the reported std across runs)
    """
    base_path = Path(base_path)
    rows: List[Dict] = []

    for es in ensemble_sizes:
        # Find all run folders for this ensemble size
        glob_pat = f"{pattern_prefix}{es}{pattern_middle}*/{file_name}"
        files = sorted(base_path.glob(glob_pat))

        vals: List[float] = []
        reps: List[float] = []

        for f in files:
            parsed = _parse_silhouette(f)
            if parsed is None:
                continue
            v, s = parsed
            # The user asked for the "absolute value before ±"; silhouette is commonly non-negative,
            # but we honor the request explicitly:
            vals.append(abs(v))
            reps.append(float(s))
            
        # Calculate uncertainty weighted mean and propagrated uncertainty
        uncertaintyWMean, uncertaintyWstd = weightedAverageUncertainty(np.array(vals), np.array(reps))
        
        minVal = np.min(np.array(vals))
        minValRep = np.max(np.array(reps)[np.array(vals)==minVal])
        
        n = len(vals)
        if n == 0:
            # still create a row so it's visible that nothing was found
            rows.append(
                dict(
                    ensemble_size=es,
                    n_runs_parsed=0,
                    value_mean=np.nan,
                    value_std_across_runs=np.nan,
                    reported_std_mean=np.nan,
                    reported_std_std=np.nan,
                    uncertaintyWMean=np.nan,
                    uncertaintyWstd=np.nan,
                    minVal=np.nan,
                    minValRep=np.nan
                )
            )
            continue

        vals_arr = np.asarray(vals, dtype=float)
        reps_arr = np.asarray(reps, dtype=float)

        rows.append(
            dict(
                ensemble_size=es,
                n_runs_parsed=n,
                value_mean=float(np.mean(vals_arr)),
                value_std_across_runs=float(np.std(vals_arr, ddof=1)) if n > 1 else 0.0,
                reported_std_mean=float(np.mean(reps_arr)),
                reported_std_std=float(np.std(reps_arr, ddof=1)) if n > 1 else 0.0,
                uncertaintyWMean=uncertaintyWMean,
                uncertaintyWstd=uncertaintyWstd,
                minVal=minVal,
                minValRep=minValRep
            )
        )

    df = pd.DataFrame(rows).sort_values("ensemble_size").reset_index(drop=True)
    return df


def makePlot(std1, std2, meanSilhouette, summary):
    """
    makePlot creates summary plot of silhouette values given
    information of multiple runs at different emsemble size.
    summarize_silhouette_by_ensemble can be used to read summarized
    ensemble runs.
    
    Parameters
    -----------
    std1
    std2
    meanSilhouette
    
    """
    # Plot mean silhouette and envelope for std 
    idxStable = 5
    fig, axes = plt.subplots(2, 2, figsize=(8, 4), width_ratios=[2,1])


    # Plot 1
    ax1 = axes[0][0]
    ax2 = axes[1][0]

    ax1.plot(summary['ensemble_size'].values,
             summary['minVal'].values,
             '-k',
             label=r"Minimum silhouette")

    ax1.plot(summary['ensemble_size'].values,
             summary['minVal'].values-summary['minValRep'].values,
             '--k',
             label=r"Minimum silhouette - $\sigma$")
    
    ax1.plot(summary['ensemble_size'].values,
             summary['value_mean'].values,
             '-r',
             label='Averaged silhouette')

    ax1.plot(summary['ensemble_size'].values[idxStable], 
              meanSilhouette[idxStable],
              'om',
              markersize=10,
              label='Stable Ensemble Threshold')


    # Create hatch regions to define Strong and Good Boundary Stability
    ax1.fill_between(summary['ensemble_size'].values,
                     0.7+0*summary['ensemble_size'].values,
                     y2=1+0*summary['ensemble_size'].values,
                     where=(0.7+0*summary['ensemble_size'].values > -np.inf),
                     facecolor='none',
                     hatch='///',
                     edgecolor='yellow',
                     alpha=0.7,
                     label='Strong Boundary Stability')

    ax1.fill_between(summary['ensemble_size'].values,
                     0.5+0*summary['ensemble_size'].values,
                     y2=0.7+0*summary['ensemble_size'].values,
                     where=(0.5+0*summary['ensemble_size'].values > -np.inf),
                     facecolor='none',
                     hatch='///',
                     edgecolor='gray',
                     alpha=0.7,
                     label='Good Boundary Stability')


    ax1.fill_between(summary['ensemble_size'].values,
                     0*summary['ensemble_size'].values,
                     y2=0.5+0*summary['ensemble_size'].values,
                     where=(0.3+0*summary['ensemble_size'].values > -np.inf),
                     facecolor='none',
                     hatch='///',
                     edgecolor='red',
                     alpha=0.7,
                     label='Week Boundary Stability')



    fig.supylabel('Area Averaged Silhouette')

    ax1.set_ylim([0,1]);
    ax1.set_xlim([np.min(summary['ensemble_size'].values),np.max(summary['ensemble_size'].values)]);


    # Plot 2
    idxStart = 2
    std1 = std1[idxStart:]
    std2 = std2[idxStart:]


    ax2.set_xlim([np.min(summary['ensemble_size'].values),np.max(summary['ensemble_size'].values)]);

    ax2.plot(summary['ensemble_size'].values[idxStart:],
              meanSilhouette[idxStart:],
              '-r',
              label='Averaged silhouette')

    ax2.plot(summary['ensemble_size'].values[idxStart:][idxStable-idxStart], 
              meanSilhouette[idxStart:][idxStable-idxStart],
              'om',
              markersize=10,
              label='Stable Ensemble Threshold')


    
    # Plot 3 (used for creating legend)
    std1 = summary['value_std_across_runs'].values
    std2 = summary['reported_std_mean'].values

    ax3 = axes[0][1]    
    ax3.plot(summary['ensemble_size'].values,
             summary['minVal'].values,
             '-k',
             label=r"Minimum silhouette")

    ax3.plot(summary['ensemble_size'].values,
             summary['minVal'].values-summary['minValRep'].values,
             '--k',
             label=r"Minimum silhouette - $\sigma$")

    ax3.plot(summary['ensemble_size'].values,
              meanSilhouette,
              '-r',
              label='Averaged silhouette')

    ax3.plot(summary['ensemble_size'].values[idxStable], 
              summary['value_mean'].values[idxStable],
              'om',
              markersize=10,
              label='Stable Ensemble Threshold')


    # Create hatch regions to define Strong and Good Boundary Stability
    ax3.fill_between(summary['ensemble_size'].values,
                     0.7+0*summary['ensemble_size'].values,
                     y2=1+0*summary['ensemble_size'].values,
                     where=(0.7+0*summary['ensemble_size'].values > -np.inf),
                     facecolor='none',
                     hatch='///',
                     edgecolor='yellow',
                     alpha=0.7,
                     label='Strong Boundary Stability')

    ax3.fill_between(summary['ensemble_size'].values,
                     0.5+0*summary['ensemble_size'].values,
                     y2=0.7+0*summary['ensemble_size'].values,
                     where=(0.5+0*summary['ensemble_size'].values > -np.inf),
                     facecolor='none',
                     hatch='///',
                     edgecolor='gray',
                     alpha=0.7,
                     label='Good Boundary Stability')


    ax3.fill_between(summary['ensemble_size'].values,
                     0*summary['ensemble_size'].values,
                     y2=0.5+0*summary['ensemble_size'].values,
                     where=(0.3+0*summary['ensemble_size'].values > -np.inf),
                     facecolor='none',
                     hatch='///',
                     edgecolor='red',
                     alpha=0.7,
                     label='Week Boundary Stability')


    
    # Formatting
    ax1.set_xticklabels([])
    ax1.set_yticks([0,0.5,0.7,1])

    ax2.set_xlabel('Ensemble Count')
    ax2.set_ylim([0.7,0.9]);

    ax3.set_ylim([-1,-.5]);
    ax3.set_xlim([0,.5]);
    ax3.legend(loc='lower center')
    ax3.axis('off')

    ax4 = axes[1][1]
    ax4.axis('off')


    fig.tight_layout()


    # Output figure
    plt.savefig(os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/EnsembleSilhouetteScores/SilhouetteEnsembleSizeAnalysis.png",
                dpi=600,
                transparent=True)
    plt.savefig(os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/EnsembleSilhouetteScores/SilhouetteEnsembleSizeAnalysis.svg",
                dpi=600,
                transparent=True)

    



# Read model outputs (i.e., readme.txt files)
base = os.getcwd()+"/figures/GMD_Manuscript/CodeOutputs/EnsembleSilhouetteScores"
ensemble_sizes = [1, 3, 5, 10, 20, 30, 40, 50]
summary = summarize_silhouette_by_ensemble(base, ensemble_sizes)
print(summary.to_string(index=False))


# Report some silhouette score information on the ensembles
std1 = summary['value_std_across_runs'].values
std2 = summary['reported_std_mean'].values
meanSilhouette = summary['value_mean'].values


std1 = summary['uncertaintyWstd'].values
std2 = summary['reported_std_mean'].values
meanSilhouette = summary['uncertaintyWMean'].values
makePlot(std1, std2, meanSilhouette, summary)



# Regional Analysis Example

In [ ]:
#################################################
#################### Imports ####################
#################################################
import os
import ExoCcycle as EC
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from netCDF4 import Dataset
import cartopy.crs as ccrs # type: ignore


#########################################
### Define Community Detection Inputs ###
#########################################
# Set the detection method
communityDetectionMethod = "Leiden"

# Define basin merging criteria
# mergerPackageName = 'None'; # ['threshold'] = [0]
mergerPackageName = 'Lite';

# Define basin merging criteria
mergerPackage = EC.utils.mergerPackages(mergerPackageName);
mergerPackage['verbose'] = False;

# Resolution for quality function
resolutions = [.01]

# Minimum number of basins to have in output (only used for
# girvan-newman or composite algorithms)
minBasinCnts = [12]

# Set the ensemble size to use for the first part of the composite community detection
# This part runs Louvain or Leiden algorithms to reduce the network complexity. Setting
# a non-one ensemble ensures that community structure is robust given inherent randomness
# of initial node clustering. Note that ensembles of size 100 for 1 degree resolution data
# only increase total computational time by 1-2 minutes.
ensembleSizes = [50];


# Set spatial resolution
spatialResolution = .1
region = np.array([[-100, 20], [-80, 31]])


# Show the resolution, ensembleSize, and minBasinCnt used for community detection
# runs. Note that only one community detection is run here.
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    print(resolution, ensembleSize, minBasinCnt)

# Run for multiple resolution, ensembleSize, and minBasinCnt
for resolution, ensembleSize, minBasinCnt in zip(resolutions, ensembleSizes, minBasinCnts):
    detectionMethod = {"method":communityDetectionMethod,
                       "resolution":resolution,
                       "minBasinCnt":minBasinCnt,
                       "ensembleSize":ensembleSize,
                       "minBasinLargerThanSmallMergers":False,
                       "mergerPackage":mergerPackage,
                       "njobs":4}

    # Set the edge weight scheme for node connections
    # Options:
    #    "useGravity", "useLogistic", "useQTGaussianShiftedGaussianWeightDistribution"
    edgeWeightMethod = {"method":"useQTGaussianShiftedGaussianWeightDistribution",
                       "shortenFactor": 5,
                       "shiftFactor": .5,
                       "minWeight": 0.01}

    # Make folder to hold figure results    
    fldName = EC.utils.makeFolderSeries(fldBase='figures/GMD_Manuscript/CodeOutputs/CompositeMethod_Leiden')
    print("Storing images in {}".format(fldName))

    # Short readme text to write to folder with images
    readmetxt = "Bathymetry used to calculate basin boundaries.";
    readmetxt += "\nUsing {0} detection with the DQT-CDF edge weighting method".format(detectionMethod["method"])
    readmetxt += "DQT-CDF parameters: 'useQTGaussianShiftedGaussianWeightDistribution' with shorten = sigma*{0} & shift = sigma*{1}, minimum weight = {2})".format(edgeWeightMethod["shortenFactor"], edgeWeightMethod["shiftFactor"], edgeWeightMethod['minWeight']);
    readmetxt += "\n{} resolution: {}".format(detectionMethod["method"], detectionMethod["resolution"]);
    readmetxt += "\n{} ensemble size: {}".format(detectionMethod["method"], detectionMethod["ensembleSize"]);
    readmetxt += "\nGirvan-Newman minimum unisolated basins: {}".format(detectionMethod['minBasinCnt']);
    readmetxt += "\nCommunity merger package is EC.utils.mergePackage(package='{}'), Chi = {}".format(mergerPackageName, ", ".join( list(mergerPackage['mergeSmallBasins']['threshold'].astype(str)) ));    


    #################################################################
    ### Create basin object and set Field for Community detection ###
    #################################################################

    # Create basin object
    body = ["Earth", "Mars", "Venus", "Moon"]
    body = body[0]

    basins = EC.utils.BasinsEA(dataDir=os.getcwd()+"/bathymetries/{}".format(body),
                             filename="{}_resampled_{:0.1f}deg.nc".format(body, spatialResolution),
                             body=body,
                             region=region  );


    # Assign fields to use in community detection
    basins.useFields(fieldList=np.array(["Field1"]))

    # Show all fields stored in basins object
    basins.getFields(usedFields = False)

    # Show all fields stored in basins object that will be used
    # for community detection.
    basins.getFields(usedFields = True)


    #########################################
    ### Run Community Detection Algorithm ###
    #########################################
    # Define basins based on user input boundaries.
    # For the Louvain-Girvan-Newman composite algorithm the variable
    # minBasinCnt refers to the number of basins to maintain that are
    # not completely isolated after running the louvain algorithm.
    basins.defineBasins(detectionMethod = detectionMethod,
                        edgeWeightMethod = edgeWeightMethod,
                        reducedRes={"on":True,"factor":1},
                        read=False,
                        write=True,
                        verbose=False)


    # Merge communities based off criteria 
    basins.applyMergeBasinMethods(mergerID=0, mergerPackage=mergerPackage)

    # Convert basinID equal area grid to regular grid
    basins.interp2regularGrid(mask=True)
    
    # Get small and large basins
    percentThreshold = 0.5;
    basinSizeDic = basins.getBasinSize(fraction=True, Threshold=percentThreshold)
    
    readmetxt += basinSizeDic["text"]
    
    # Get community detection metrics
    metrics, metricText = basins.reportEvaluationMetrics(returnText=True,
                                                         resolution=resolution,
                                                         ensembleSize=ensembleSize,
                                                         distance_threshold=0.3)
    readmetxt += ("\n\n" + metricText)
    
    
    ##################################
    ### Calculate Silhouette field ###
    ##################################
    silhouette = basins.interp2regularGrid(propertyName="consensus_silhouette", mask=True)

    # Calculate area weighted average and standard deviation (for plotting)
    areaWeights, longitudes, latitudes, totalArea, totalAreaCalculated = EC.utils.areaWeights(resolution=basins.Fields["Field1"]['resolution'],
                                                                                              LonStEd = [np.min(basins.lon),np.max(basins.lon)+basins.Fields["Field1"]['resolution']/2],
                                                                                              LatStEd = [np.min(basins.lat),np.max(basins.lat)+basins.Fields["Field1"]['resolution']/2])
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)
    aveSilhouette, stdSilhouette = EC.utils.weightedAvgAndStd(silhouette, areaWeights)
    print( "Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette) )
    readmetxt += ("Silhouette value: {0:2.4f} \u00B1 {1:2.4f}".format(aveSilhouette, stdSilhouette))  
    
    
    ###########################################
    ### Report community evaluation metrics ###
    ###########################################
    with open(fldName+"/readme.txt", "w") as text_file:
        text_file.write(readmetxt)



    #####################################
    ### Plot results of community IDs ###
    #####################################


    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_Contour.png",
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,np.nanmax(basins.BasinIDA)]},
                        pltOpts={"valueType": "BasinID divided by {}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "-",
                                 "plotTitle":"",
                                 "region": [region[0,0],region[1,0], region[0,1],region[1,1]],
                                 "projection":ccrs.Mercator(),
                                 "mesh":False,
                                 "coastlines":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotZeroContour":False,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)  



    # Calculate area weighted average and standard deviation (for plotting)
    ave, std = EC.utils.weightedAvgAndStd(basins.bathymetry, areaWeights)

    #########################
    ### Plot input fields ###
    #########################
    EC.plotHelper.plotGlobal(basins.lat, basins.lon, basins.bathymetry,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[ave-1*std,
                                                ave+1*std]},
                        pltOpts={"valueType": "{0}".format(basins.Fields["Field1"]['parameterName']),
                                 "valueUnits": "{}".format(basins.Fields["Field1"]['parameterUnit']),
                                 "plotTitle":"",
                                 "region": [region[0,0],region[1,0], region[0,1],region[1,1]],
                                 "projection":ccrs.Mercator(),
                                 "plotZeroContour":False,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    
    ##############################
    ### Plot Silhouette fields ###
    ##############################
    EC.plotHelper.plotGlobalSilhouette(basins.lat, basins.lon, silhouette, basins.BasinIDA,
                        outputDir = os.getcwd()+"/"+fldName,
                        fidName = "plotGlobal_silhouette_1_{0}.png".format(basins.Fields["Field1"]['parameterName']),
                        cmapOpts={"cmap":"jet",
                                  "cbar-title":"cbar-title",
                                  "cbar-range":[0,
                                                1],
                                  "cbar-levels":np.array([0,.25, .5, .7, 1]),
                                  "cbar-intervals":np.array([0,.25, .5, .7, 1]),
                                  "cbar-level-names":["No", "Weak", "Medium", "Strong"]},
                        pltOpts={"valueType": "{0}".format("Silhouette Structure"),
                                 "valueUnits": "{}".format("-"),
                                 "plotTitle":"",
                                 "region": [region[0,0],region[1,0], region[0,1],region[1,1]],
                                 "projection":ccrs.Mercator(),
                                 "plotZeroContour":False,
                                 "nanSolidPoly":True,
                                 "nanSolidPolyOutline":True,
                                 "plotIntegerContours":True,
                                 "transparent":True},
                        savePNG=True,
                        saveSVG=False)
    

